In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import (
    StratifiedKFold, LeaveOneOut, cross_val_score, 
    GridSearchCV, cross_validate
)
from sklearn.feature_selection import (
    SelectKBest, f_classif, mutual_info_classif,
    RFE, SelectFromModel, VarianceThreshold
)
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, roc_auc_score, matthews_corrcoef,
    confusion_matrix, classification_report, make_scorer
)
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.decomposition import PCA, KernelPCA
from sklearn.utils import resample
from collections import Counter
from scipy import stats
import warnings
import pickle
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

In [59]:
#matrix_data = matrix_data.drop(index=0) 

In [179]:
#matrix_data.head()

In [181]:
#bgx.head()

In [182]:
#matrix_data.shape

In [183]:
#matrix_data.describe()

In [184]:
#bgx.head()

In [186]:
#bgx.shape

In [187]:
#bgx.head(2)

In [188]:
#print(matrix_data.index.name)

In [191]:
#matrix_data.index.name

In [192]:
#matrix_data.head(2)

In [196]:
#merged_data.head(2)

In [74]:
# merged_data[['Symbol', 'var']].head(2)

In [201]:
#collapse_cols.head()

In [34]:
# print(normal_data.shape)
# print(normal_data.index[:5])
# print(normal_data.columns[:5])

(42, 25159)
Index(['GSM1318948', 'GSM1318949', 'GSM1318950', 'GSM1318951', 'GSM1318952'], dtype='object')
Index(['OR6C68', 'SMC1B', 'LOC652184', 'SMR3A', 'LCE1D'], dtype='object', name='Symbol')


In [15]:
#normal_data.shape

In [160]:
#normal_data.head()

In [159]:
#print(normal_data.shape)
#normal_data.head()

In [206]:
#matrix_data.columns

with open("data\\GSE54564_series_matrix.txt", 'r') as f:
    for line in f:
        if "characteristics_ch1" in line:
            print(line.strip())

In [244]:
#print(labels)
#print(len(labels))
#labels[:5]

In [36]:
# number = pd.Series(labels).value_counts()

In [243]:
#number

In [35]:
# print(y.value_counts())

In [323]:
#print(ttest)

In [5]:
def load_gse54564_data(data1_path, data2_path, data3_path):
    """Load GSE54564 data"""
    
    print("Loading GSE54564 data...")
    
    matrix_data = pd.read_csv(data2_path, sep="\t", comment="!", index_col=0)
    print(f"Expression matrix: {matrix_data.shape}")
    
    labels = []
    with open(data2_path, 'r') as f:
        for line in f:
            if "disease state" in line.lower():
                labels = line.strip().split("\t")[1:]
                break
    
    y = (pd.Series(labels, index=matrix_data.columns)
         .str.replace('"', '')
         .str.replace('disease state:', "")
         .str.strip()
         .map({"MDD case": 1, "Control": 0}))
    
    print(f"Labels: {y.value_counts().to_dict()}")
    
    with open(data3_path, 'r') as f:
        lines = f.readlines()
    
    start = None
    for i, line in enumerate(lines):
        if line.strip() == "[Probes]":
            start = i + 1
            break
    
    bgx = pd.read_csv(data3_path, sep="\t", skiprows=start, engine="python")
    print(f"BGX annotation: {bgx.shape}")
    
    probe_to_gene = {}
    if 'Probe_Id' in bgx.columns and 'Symbol' in bgx.columns:
        probe_to_gene = dict(zip(bgx['Probe_Id'], bgx['Symbol']))
    elif 'Name' in bgx.columns and 'Symbol' in bgx.columns:
        probe_to_gene = dict(zip(bgx['Name'], bgx['Symbol']))
    
    gene_names = []
    valid_probes = []
    for probe in matrix_data.index:
        if probe in probe_to_gene and pd.notna(probe_to_gene[probe]):
            gene_names.append(probe_to_gene[probe])
            valid_probes.append(probe)
    
    X = matrix_data.loc[valid_probes].T.values
    
    print(f"Final shape: X={X.shape}, y={y.shape}, genes={len(gene_names)}")
    
    return X, y.values, gene_names, probe_to_gene

In [6]:
def preprocess_expression_data(X, gene_names=None, remove_low_var=True, var_threshold=0.1):
    """Preprocess expression data with log2 transform, z-score normalization, and variance filtering"""
    
    print("\nPreprocessing expression data...")
    print(f"Input: {X.shape}, gene_names: {len(gene_names) if gene_names else None}")
    
    if np.max(X) > 100:
        X = np.log2(X + 1)
    
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X)
    
    if remove_low_var:
        gene_variances = np.var(X_normalized, axis=0)
        high_var_mask = gene_variances > var_threshold
        
        n_before = X_normalized.shape[1]
        n_after = np.sum(high_var_mask)
        
        print(f"Genes: {n_before} -> {n_after} (removed {n_before - n_after})")
        
        X_filtered = X_normalized[:, high_var_mask]
        
        if gene_names is not None:
            if len(gene_names) != n_before:
                print(f"WARNING: gene_names length ({len(gene_names)}) != features ({n_before})")
                gene_names = gene_names[:n_before]
            
            gene_names_filtered = [gene_names[i] for i in range(len(gene_names)) if high_var_mask[i]]
            
            assert len(gene_names_filtered) == X_filtered.shape[1], \
                f"Mismatch: {len(gene_names_filtered)} names vs {X_filtered.shape[1]} features"
            
            return X_filtered, gene_names_filtered
        
        return X_filtered, None
    
    return X_normalized, gene_names


In [7]:
data1 = "../data/GSE54564_non-normalized.txt"
data2 = "../data/GSE54564_series_matrix.txt"
data3 = "../data/NCBI_Depression.bgx"

print("=" * 80)
print("STEP 1: LOADING DATA")
print("=" * 80)
X_raw, y, gene_names_raw, probe_mapping = load_gse54564_data(data1, data2, data3)

with open('output/Phase_2/raw_data.pkl', 'wb') as f:
    pickle.dump({
        'X_raw': X_raw,
        'y': y,
        'gene_names_raw': gene_names_raw,
        'probe_mapping': probe_mapping
    }, f)
print("Raw data saved to 'output/Phase_2/raw_data.pkl'")

print("\n" + "=" * 80)
print("STEP 2: PREPROCESSING")
print("=" * 80)
X_processed, gene_names_filtered = preprocess_expression_data(X_raw, gene_names_raw)

print(f"\nVerification:")
print(f"  X_processed shape: {X_processed.shape}")
print(f"  gene_names_filtered length: {len(gene_names_filtered) if gene_names_filtered else None}")
print(f"  Match: {len(gene_names_filtered) == X_processed.shape[1] if gene_names_filtered else False}")

with open('output/Phase_2/processed_data.pkl', 'wb') as f:
    pickle.dump({
        'X_processed': X_processed,
        'y': y,
        'gene_names_filtered': gene_names_filtered
    }, f)
print("Processed data saved to 'output/Phase_2/processed_data.pkl'")

print("\n" + "=" * 80)
print("DATA LOADING COMPLETE")
print("=" * 80)

STEP 1: LOADING DATA
Loading GSE54564 data...
Expression matrix: (48803, 42)
Labels: {1: 21, 0: 21}
BGX annotation: (49617, 28)
Final shape: X=(42, 36157), y=(42,), genes=36157
Raw data saved to 'output/Phase_2/raw_data.pkl'

STEP 2: PREPROCESSING

Preprocessing expression data...
Input: (42, 36157), gene_names: 36157
Genes: 36157 -> 36157 (removed 0)

Verification:
  X_processed shape: (42, 36157)
  gene_names_filtered length: 36157
  Match: True
Processed data saved to 'output/Phase_2/processed_data.pkl'

DATA LOADING COMPLETE


In [8]:
class StableGeneSelector:
    """Robust feature selection combining multiple strategies with bootstrap stability"""
    
    def __init__(self, n_genes_to_select=50, n_bootstrap=100, stability_threshold=0.5, random_state=42):
        self.n_genes_to_select = n_genes_to_select
        self.n_bootstrap = n_bootstrap
        self.stability_threshold = stability_threshold
        self.random_state = random_state
        self.selected_genes_ = None
        self.gene_scores_ = None
        
    def fit(self, X, y):
        """Identify stable genes using multiple selection strategies"""
        np.random.seed(self.random_state)
        n_samples, n_features = X.shape
        
        l1_scores = np.zeros(n_features)
        univariate_scores = np.zeros(n_features)
        elastic_scores = np.zeros(n_features)
        
        print(f"  Running {self.n_bootstrap} bootstrap iterations for stability selection...")
        
        for i in tqdm(range(self.n_bootstrap), desc=f"  StableGeneSelector (n={self.n_bootstrap})", leave=False):
            idx = resample(np.arange(n_samples), n_samples=int(0.7 * n_samples), random_state=i)
            X_boot = X[idx]
            y_boot = y[idx]
            
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_boot)
            
            C_value = np.random.choice([0.01, 0.05, 0.1, 0.2])
            try:
                l1_model = LogisticRegression(penalty='l1', C=C_value, solver='liblinear',
                                             max_iter=1000, random_state=i)
                l1_model.fit(X_scaled, y_boot)
                selected_l1 = np.abs(l1_model.coef_[0]) > 1e-6
                l1_scores += selected_l1
            except:
                pass
            
            selector = SelectKBest(f_classif, k=min(self.n_genes_to_select * 2, n_features))
            selector.fit(X_boot, y_boot)
            univariate_scores += selector.get_support()
            
            if n_samples > 20:
                try:
                    elastic = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.7,
                                                C=0.1, max_iter=5000, random_state=i)
                    elastic.fit(X_scaled, y_boot)
                    selected_elastic = np.abs(elastic.coef_[0]) > 1e-6
                    elastic_scores += selected_elastic
                except:
                    pass
        
        l1_scores = l1_scores / self.n_bootstrap
        univariate_scores = univariate_scores / self.n_bootstrap
        elastic_scores = elastic_scores / self.n_bootstrap
        
        if n_samples > 20:
            combined_scores = 0.4 * l1_scores + 0.3 * univariate_scores + 0.3 * elastic_scores
        else:
            combined_scores = 0.6 * l1_scores + 0.4 * univariate_scores
        
        stable_mask = combined_scores >= self.stability_threshold
        stable_indices = np.where(stable_mask)[0]
        
        if len(stable_indices) < self.n_genes_to_select:
            stable_indices = np.argsort(combined_scores)[-self.n_genes_to_select:]
        elif len(stable_indices) > self.n_genes_to_select * 2:
            top_indices = np.argsort(combined_scores[stable_indices])[-self.n_genes_to_select:]
            stable_indices = stable_indices[top_indices]
        
        self.selected_genes_ = stable_indices
        self.gene_scores_ = combined_scores
        
        print(f"  Selected {len(self.selected_genes_)} stable genes")
        return self
    
    def transform(self, X):
        return X[:, self.selected_genes_]
    
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X)

In [9]:
def nested_cv_univariate_svm(X, y, random_state=42):
    """Nested CV with univariate feature selection and RBF-SVM"""
    
    print("\nAPPROACH 1: Nested CV with Univariate Feature Selection + RBF-SVM")
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif)),
        ('classifier', SVC(kernel='rbf', probability=True))
    ])
    
    param_grid = {
        'selector__k': [20, 50, 100, 200, 500],
        'classifier__C': [0.01, 0.1, 1, 10, 100],
        'classifier__gamma': ['scale', 0.001, 0.01, 0.1]
    }
    
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    outer_scores = {'accuracy': [], 'roc_auc': [], 'mcc': [], 'best_params': []}
    
    n_splits = outer_cv.get_n_splits()
    fold_iterator = tqdm(outer_cv.split(X, y), desc="  Outer Folds", total=n_splits)
    
    for fold_idx, (train_idx, test_idx) in enumerate(fold_iterator):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        grid_search = GridSearchCV(pipe, param_grid, cv=inner_cv,
                                  scoring='roc_auc', n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        y_pred_proba = best_model.predict_proba(X_test)[:, 1]
        
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)
        mcc = matthews_corrcoef(y_test, y_pred)
        
        outer_scores['accuracy'].append(acc)
        outer_scores['roc_auc'].append(auc)
        outer_scores['mcc'].append(mcc)
        outer_scores['best_params'].append(grid_search.best_params_)
        
        fold_iterator.set_postfix(auc=f"{auc:.3f}", mcc=f"{mcc:.3f}")
    
    print(f"\n  Mean Accuracy: {np.mean(outer_scores['accuracy']):.3f} ± {np.std(outer_scores['accuracy']):.3f}")
    print(f"  Mean ROC-AUC: {np.mean(outer_scores['roc_auc']):.3f} ± {np.std(outer_scores['roc_auc']):.3f}")
    print(f"  Mean MCC: {np.mean(outer_scores['mcc']):.3f} ± {np.std(outer_scores['mcc']):.3f}")
    
    return outer_scores


In [10]:
def elastic_net_feature_selection_pipeline(X, y, random_state=42):
    """L1 (Lasso) feature selection with non-linear classifier"""
    
    print("\nAPPROACH 2: L1 (Lasso) Feature Selection + Non-linear Classifier")
    
    l1_selector = LinearSVC(penalty='l1', dual=False, max_iter=5000, random_state=random_state)
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('feature_selector', SelectFromModel(l1_selector)),
        ('classifier', SVC(kernel='rbf', probability=True))
    ])
    
    param_grid = {
        'feature_selector__estimator__C': [0.01, 0.1, 1.0],
        'feature_selector__threshold': ['0.5*mean', 'mean', '1.25*mean'],
        'classifier__C': [0.1, 1, 10],
        'classifier__gamma': ['scale']
    }
    
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    outer_scores = {'accuracy': [], 'roc_auc': [], 'mcc': [], 'n_features': []}
    
    n_splits = outer_cv.get_n_splits()
    fold_iterator = tqdm(outer_cv.split(X, y), desc="  Outer Folds", total=n_splits)
    
    for fold_idx, (train_idx, test_idx) in enumerate(fold_iterator):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        grid_search = GridSearchCV(pipe, param_grid, cv=inner_cv,
                                  scoring='roc_auc', n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        y_pred_proba = best_model.predict_proba(X_test)[:, 1]
        
        n_features = np.sum(best_model.named_steps['feature_selector'].get_support())
        
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)
        mcc = matthews_corrcoef(y_test, y_pred)
        
        outer_scores['accuracy'].append(acc)
        outer_scores['roc_auc'].append(auc)
        outer_scores['mcc'].append(mcc)
        outer_scores['n_features'].append(n_features)
        
        fold_iterator.set_postfix(auc=f"{auc:.3f}", n_feat=n_features)
    
    print(f"\n  Mean Accuracy: {np.mean(outer_scores['accuracy']):.3f} ± {np.std(outer_scores['accuracy']):.3f}")
    print(f"  Mean ROC-AUC: {np.mean(outer_scores['roc_auc']):.3f} ± {np.std(outer_scores['roc_auc']):.3f}")
    print(f"  Mean MCC: {np.mean(outer_scores['mcc']):.3f} ± {np.std(outer_scores['mcc']):.3f}")
    print(f"  Mean Features: {np.mean(outer_scores['n_features']):.0f} ± {np.std(outer_scores['n_features']):.0f}")
    
    return outer_scores


In [11]:
def stability_selection_pipeline(X, y, n_bootstrap=50, threshold=0.6, random_state=42):
    """Stability selection for robust feature identification"""
    
    print("\nAPPROACH 3: Stability Selection + RBF-SVM")
    
    selector = StableGeneSelector(n_genes_to_select=50, n_bootstrap=n_bootstrap,
                                  stability_threshold=threshold, random_state=random_state)
    X_selected = selector.fit_transform(X, y)
    
    print(f"\n  Evaluating classifier on {X_selected.shape[1]} stable features...")
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', SVC(kernel='rbf', probability=True))
    ])
    
    param_grid = {
        'classifier__C': [0.01, 0.1, 1, 10, 100],
        'classifier__gamma': ['scale', 0.001, 0.01, 0.1]
    }
    
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    outer_scores = {'accuracy': [], 'roc_auc': [], 'mcc': []}
    
    n_splits = outer_cv.get_n_splits()
    fold_iterator = tqdm(outer_cv.split(X_selected, y), desc="  Evaluation Folds", total=n_splits)
    
    for fold_idx, (train_idx, test_idx) in enumerate(fold_iterator):
        X_train, X_test = X_selected[train_idx], X_selected[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        grid_search = GridSearchCV(pipe, param_grid, cv=inner_cv,
                                  scoring='roc_auc', n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        y_pred_proba = best_model.predict_proba(X_test)[:, 1]
        
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)
        mcc = matthews_corrcoef(y_test, y_pred)
        
        outer_scores['accuracy'].append(acc)
        outer_scores['roc_auc'].append(auc)
        outer_scores['mcc'].append(mcc)
        
        fold_iterator.set_postfix(auc=f"{auc:.3f}", mcc=f"{mcc:.3f}")
    
    print(f"\n  Mean Accuracy: {np.mean(outer_scores['accuracy']):.3f} ± {np.std(outer_scores['accuracy']):.3f}")
    print(f"  Mean ROC-AUC: {np.mean(outer_scores['roc_auc']):.3f} ± {np.std(outer_scores['roc_auc']):.3f}")
    print(f"  Mean MCC: {np.mean(outer_scores['mcc']):.3f} ± {np.std(outer_scores['mcc']):.3f}")
    
    return outer_scores, selector.selected_genes_, selector.gene_scores_


In [12]:
def mutual_information_rfe_pipeline(X, y, random_state=42):
    """Two-stage feature selection: MI pre-filter + RFE fine-tuning"""
    
    print("\nAPPROACH 4: Mutual Information + RFE Pipeline")
    
    class MIRFEPipeline:
        def __init__(self, n_mi_features=1000, n_rfe_features=50):
            self.n_mi_features = n_mi_features
            self.n_rfe_features = n_rfe_features
            self.scaler = StandardScaler()
            self.mi_selector = SelectKBest(mutual_info_classif, k=n_mi_features)
            self.rfe_selector = None
            self.classifier = SVC(kernel='rbf', probability=True)
            
        def fit(self, X, y):
            X_scaled = self.scaler.fit_transform(X)
            X_mi = self.mi_selector.fit_transform(X_scaled, y)
            
            if X_mi.shape[1] > self.n_rfe_features:
                self.rfe_selector = RFE(LinearSVC(max_iter=5000, random_state=42),
                                       n_features_to_select=self.n_rfe_features, step=0.1)
                X_rfe = self.rfe_selector.fit_transform(X_mi, y)
            else:
                X_rfe = X_mi
                
            self.classifier.fit(X_rfe, y)
            return self
            
        def predict(self, X):
            X_scaled = self.scaler.transform(X)
            X_mi = self.mi_selector.transform(X_scaled)
            if self.rfe_selector is not None:
                X_rfe = self.rfe_selector.transform(X_mi)
            else:
                X_rfe = X_mi
            return self.classifier.predict(X_rfe)
            
        def predict_proba(self, X):
            X_scaled = self.scaler.transform(X)
            X_mi = self.mi_selector.transform(X_scaled)
            if self.rfe_selector is not None:
                X_rfe = self.rfe_selector.transform(X_mi)
            else:
                X_rfe = X_mi
            return self.classifier.predict_proba(X_rfe)
    
    configs = [(1000, 50), (500, 30), (2000, 100)]
    best_config = None
    best_score = -1
    
    print("  Testing feature configurations...")
    
    for n_mi, n_rfe in tqdm(configs, desc="  Config Testing"):
        pipeline = MIRFEPipeline(n_mi_features=min(n_mi, X.shape[1]), n_rfe_features=n_rfe)
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
        scores = []
        
        for train_idx, test_idx in cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            pipeline.fit(X_train, y_train)
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            scores.append(roc_auc_score(y_test, y_pred_proba))
        
        mean_score = np.mean(scores)
        print(f"    MI={n_mi}, RFE={n_rfe}: ROC-AUC={mean_score:.3f}")
        
        if mean_score > best_score:
            best_score = mean_score
            best_config = (n_mi, n_rfe)
    
    print(f"\n  Best configuration: MI={best_config[0]}, RFE={best_config[1]}")
    
    pipeline = MIRFEPipeline(n_mi_features=min(best_config[0], X.shape[1]), n_rfe_features=best_config[1])
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    outer_scores = {'accuracy': [], 'roc_auc': [], 'mcc': []}
    
    n_splits = cv.get_n_splits()
    fold_iterator = tqdm(cv.split(X, y), desc="  Final Evaluation", total=n_splits)
    
    for train_idx, test_idx in fold_iterator:
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
        
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)
        mcc = matthews_corrcoef(y_test, y_pred)
        
        outer_scores['accuracy'].append(acc)
        outer_scores['roc_auc'].append(auc)
        outer_scores['mcc'].append(mcc)
        
        fold_iterator.set_postfix(auc=f"{auc:.3f}")
    
    print(f"\n  Mean Accuracy: {np.mean(outer_scores['accuracy']):.3f} ± {np.std(outer_scores['accuracy']):.3f}")
    print(f"  Mean ROC-AUC: {np.mean(outer_scores['roc_auc']):.3f} ± {np.std(outer_scores['roc_auc']):.3f}")
    print(f"  Mean MCC: {np.mean(outer_scores['mcc']):.3f} ± {np.std(outer_scores['mcc']):.3f}")
    
    return outer_scores, best_config

In [13]:
def ensemble_feature_bagging(X, y, n_estimators=50, feature_fraction=0.1, random_state=42):
    """Ensemble of models trained on random feature subsets"""
    
    print("\nADVANCED: Feature Bagging Ensemble")
    
    np.random.seed(random_state)
    n_features = X.shape[1]
    n_features_per_model = int(n_features * feature_fraction)
    
    print(f"  Training {n_estimators} models with {n_features_per_model} features each...")
    
    feature_subsets = []
    for i in range(n_estimators):
        feature_indices = np.random.choice(n_features, n_features_per_model, replace=False)
        feature_subsets.append(feature_indices)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    scores = {'accuracy': [], 'roc_auc': [], 'mcc': []}
    
    n_splits = cv.get_n_splits()
    fold_iterator = tqdm(cv.split(X, y), desc="  Outer Folds", total=n_splits)
    
    for train_idx, test_idx in fold_iterator:
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        fold_models = []
        
        for feature_indices in tqdm(feature_subsets, desc="    Training Ensemble", leave=False):
            X_subset = X_train[:, feature_indices]
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_subset)
            
            model = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale')
            model.fit(X_scaled, y_train)
            
            fold_models.append({'model': model, 'scaler': scaler, 'features': feature_indices})
        
        predictions_proba = []
        for model_info in fold_models:
            X_test_subset = X_test[:, model_info['features']]
            X_test_scaled = model_info['scaler'].transform(X_test_subset)
            pred_proba = model_info['model'].predict_proba(X_test_scaled)[:, 1]
            predictions_proba.append(pred_proba)
        
        y_pred_proba = np.mean(predictions_proba, axis=0)
        y_pred = (y_pred_proba > 0.5).astype(int)
        
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)
        mcc = matthews_corrcoef(y_test, y_pred)
        
        scores['accuracy'].append(acc)
        scores['roc_auc'].append(auc)
        scores['mcc'].append(mcc)
        
        fold_iterator.set_postfix(auc=f"{auc:.3f}")
    
    print(f"\n  Mean Accuracy: {np.mean(scores['accuracy']):.3f} ± {np.std(scores['accuracy']):.3f}")
    print(f"  Mean ROC-AUC: {np.mean(scores['roc_auc']):.3f} ± {np.std(scores['roc_auc']):.3f}")
    print(f"  Mean MCC: {np.mean(scores['mcc']):.3f} ± {np.std(scores['mcc']):.3f}")
    
    return scores

In [14]:
def optimal_nested_cv_pipeline(X, y, n_genes=50, random_state=42):
    """Nested Leave-One-Out CV with stability selection"""
    
    print("\nOPTIMAL PIPELINE: Nested Leave-One-Out CV with Stability Selection")
    
    outer_cv = LeaveOneOut()
    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
    
    results = {
        'predictions': [],
        'probabilities': [],
        'true_labels': [],
        'selected_genes_per_fold': []
    }
    
    print(f"  Running Leave-One-Out CV ({X.shape[0]} iterations)...")
    
    fold_iterator = tqdm(outer_cv.split(X, y), desc="  LOO-CV Folds", total=X.shape[0])
    
    param_combos = list(itertools.product(
        [0.01, 0.1, 1.0, 10.0],
        ['scale', 0.01, 0.1]
    ))
    
    for i, (train_idx, test_idx) in enumerate(fold_iterator):
        X_train_outer, X_test_outer = X[train_idx], X[test_idx]
        y_train_outer, y_test_outer = y[train_idx], y[test_idx]
        
        selector = StableGeneSelector(n_genes_to_select=n_genes, n_bootstrap=5,
                                     stability_threshold=0.3, random_state=i)
        X_train_selected = selector.fit_transform(X_train_outer, y_train_outer)
        X_test_selected = selector.transform(X_test_outer)
        
        best_score = -1
        best_params = {'C': 1.0, 'gamma': 'scale'}
        
        param_iterator = tqdm(param_combos, desc="    Hyperparameter Tuning", leave=False)
        
        for C, gamma in param_iterator:
            scores = []
            
            for train_inner, val_inner in inner_cv.split(X_train_selected, y_train_outer):
                X_train_inner = X_train_selected[train_inner]
                y_train_inner = y_train_outer[train_inner]
                X_val_inner = X_train_selected[val_inner]
                y_val_inner = y_train_outer[val_inner]
                
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train_inner)
                X_val_scaled = scaler.transform(X_val_inner)
                
                clf = SVC(kernel='rbf', C=C, gamma=gamma, probability=True)
                clf.fit(X_train_scaled, y_train_inner)
                
                val_prob = clf.predict_proba(X_val_scaled)[:, 1]
                score = roc_auc_score(y_val_inner, val_prob)
                scores.append(score)
            
            mean_score = np.mean(scores)
            if mean_score > best_score:
                best_score = mean_score
                best_params = {'C': C, 'gamma': gamma}
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)
        
        final_clf = SVC(kernel='rbf', C=best_params['C'], gamma=best_params['gamma'], probability=True)
        final_clf.fit(X_train_scaled, y_train_outer)
        
        pred = final_clf.predict(X_test_scaled)[0]
        prob = final_clf.predict_proba(X_test_scaled)[0, 1]
        
        results['predictions'].append(pred)
        results['probabilities'].append(prob)
        results['true_labels'].append(y_test_outer[0])
        results['selected_genes_per_fold'].append(selector.selected_genes_)
    
    predictions = np.array(results['predictions'])
    probabilities = np.array(results['probabilities'])
    true_labels = np.array(results['true_labels'])
    
    accuracy = accuracy_score(true_labels, predictions)
    auc = roc_auc_score(true_labels, probabilities)
    mcc = matthews_corrcoef(true_labels, predictions)
    cm = confusion_matrix(true_labels, predictions)
    
    print(f"\n  Accuracy: {accuracy:.3f}")
    print(f"  ROC-AUC: {auc:.3f}")
    print(f"  MCC: {mcc:.3f}")
    print(f"  Confusion Matrix: TN={cm[0,0]}, FP={cm[0,1]}, FN={cm[1,0]}, TP={cm[1,1]}")
    print(f"  Sensitivity: {cm[1,1]/(cm[1,0]+cm[1,1]):.3f}")
    print(f"  Specificity: {cm[0,0]/(cm[0,0]+cm[0,1]):.3f}")
    
    gene_selection_counts = np.zeros(X.shape[1])
    for genes in results['selected_genes_per_fold']:
        gene_selection_counts[genes] += 1
    
    consistent_genes = np.where(gene_selection_counts > (X.shape[0] * 0.5))[0]
    print(f"  Consistently selected genes (>50% folds): {len(consistent_genes)}")
    
    results['metrics'] = {
        'accuracy': accuracy,
        'auc': auc,
        'mcc': mcc,
        'confusion_matrix': cm,
        'consistent_genes': consistent_genes,
        'gene_selection_frequency': gene_selection_counts / X.shape[0]
    }
    
    return results

In [15]:
try:
    X_processed
    print("Using data from memory")
except NameError:
    print("Loading from disk...")
    with open('output/Phase_2/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
        X_processed = data['X_processed']
        y = data['y']
        gene_names_filtered = data['gene_names_filtered']

print("\nAPPROACH 1: Univariate Feature Selection + RBF-SVM")
print(f"Dataset: {X_processed.shape[0]} samples × {X_processed.shape[1]} genes")
print(f"Method: ANOVA F-test + Nested CV with hyperparameter tuning")

results_univariate = nested_cv_univariate_svm(X_processed, y)

with open('output/Phase_2/results_approach1.pkl', 'wb') as f:
    pickle.dump(results_univariate, f)

print("\nAPPROACH 1 RESULTS SUMMARY")
print("Overall Performance:")
print(f"  Accuracy:     {np.mean(results_univariate['accuracy']):.3f} ± {np.std(results_univariate['accuracy']):.3f}")
print(f"  ROC-AUC:      {np.mean(results_univariate['roc_auc']):.3f} ± {np.std(results_univariate['roc_auc']):.3f}")
print(f"  MCC:          {np.mean(results_univariate['mcc']):.3f} ± {np.std(results_univariate['mcc']):.3f}")

print("\nPer-Fold Breakdown:")
for fold_idx in range(len(results_univariate['accuracy'])):
    print(f"  Fold {fold_idx+1}: Acc={results_univariate['accuracy'][fold_idx]:.3f} | "
          f"AUC={results_univariate['roc_auc'][fold_idx]:.3f} | "
          f"MCC={results_univariate['mcc'][fold_idx]:.3f}")

print("\nHyperparameter Analysis:")
param_k = [p['selector__k'] for p in results_univariate['best_params']]
param_C = [p['classifier__C'] for p in results_univariate['best_params']]
param_gamma = [p['classifier__gamma'] for p in results_univariate['best_params']]

print(f"  Features (k):  {np.mean(param_k):.0f} ± {np.std(param_k):.0f} (range: {min(param_k)}-{max(param_k)})")
print(f"  C values:      Most common = {Counter(param_C).most_common(1)[0][0]}")
print(f"  Gamma values:  Most common = {Counter(param_gamma).most_common(1)[0][0]}")

print("\nPerformance Interpretation:")
mean_auc = np.mean(results_univariate['roc_auc'])
if mean_auc >= 0.8:
    print("  EXCELLENT performance (AUC >= 0.8)")
elif mean_auc >= 0.7:
    print("  GOOD performance (0.7 <= AUC < 0.8)")
elif mean_auc >= 0.6:
    print("  FAIR performance (0.6 <= AUC < 0.7)")
else:
    print("  POOR performance (AUC < 0.6)")

approach1_detailed = pd.DataFrame({
    'Fold': range(1, len(results_univariate['accuracy']) + 1),
    'Accuracy': results_univariate['accuracy'],
    'ROC_AUC': results_univariate['roc_auc'],
    'MCC': results_univariate['mcc'],
    'Best_k': param_k,
    'Best_C': param_C,
    'Best_gamma': [str(g) for g in param_gamma]
})
approach1_detailed.to_csv('output/Phase_2/approach1_detailed.csv', index=False)

approach1_summary = pd.DataFrame({
    'Metric': ['Accuracy', 'ROC-AUC', 'MCC', 'Avg_Features'],
    'Mean': [np.mean(results_univariate['accuracy']), np.mean(results_univariate['roc_auc']),
             np.mean(results_univariate['mcc']), np.mean(param_k)],
    'Std': [np.std(results_univariate['accuracy']), np.std(results_univariate['roc_auc']),
            np.std(results_univariate['mcc']), np.std(param_k)],
    'Min': [np.min(results_univariate['accuracy']), np.min(results_univariate['roc_auc']),
            np.min(results_univariate['mcc']), min(param_k)],
    'Max': [np.max(results_univariate['accuracy']), np.max(results_univariate['roc_auc']),
            np.max(results_univariate['mcc']), max(param_k)]
})
approach1_summary.to_csv('output/Phase_2/approach1_summary.csv', index=False)

print("\nResults saved:")
print("  output/Phase_2/results_approach1.pkl")
print("  output/Phase_2/approach1_detailed.csv")
print("  output/Phase_2/approach1_summary.csv")

Using data from memory

APPROACH 1: Univariate Feature Selection + RBF-SVM
Dataset: 42 samples × 36157 genes
Method: ANOVA F-test + Nested CV with hyperparameter tuning

APPROACH 1: Nested CV with Univariate Feature Selection + RBF-SVM


  Outer Folds:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits

  Mean Accuracy: 0.525 ± 0.061
  Mean ROC-AUC: 0.685 ± 0.074
  Mean MCC: 0.072 ± 0.101

APPROACH 1 RESULTS SUMMARY
Overall Performance:
  Accuracy:     0.525 ± 0.061
  ROC-AUC:      0.685 ± 0.074
  MCC:          0.072 ± 0.101

Per-Fold Breakdown:
  Fold 1: Acc=0.556 | AUC=0.600 | MCC=0.100
  Fold 2: Acc=0.444 | AUC=0.700 | MCC=0.000
  Fold 3: Acc=0.500 | AUC=0.688 | MCC=0.000
  Fold 4: Acc=0.500 | AUC=0.625 | MCC=0.000
  Fold 5: Acc=0.625 | AUC=0.812 | MCC=0.258

Hyperparameter Analysis:
  Features (k):  68 ± 67 (range: 20-200)
  C values:      Most common = 100
  Gamma values:  Most common = 0.001

Performance Interpretation:
  FAIR performance (0.6 <= AUC < 0.7)

Results sa

In [16]:
try:
    X_processed
    print("Using data from memory")
except NameError:
    print("Loading from disk...")
    with open('output/Phase_2/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
        X_processed = data['X_processed']
        y = data['y']
        gene_names_filtered = data['gene_names_filtered']

print("\nAPPROACH 2: L1 (Lasso) Feature Selection + RBF-SVM")
print(f"Dataset: {X_processed.shape[0]} samples × {X_processed.shape[1]} genes")
print(f"Method: LinearSVC L1 penalty + Nested CV")

results_l1 = elastic_net_feature_selection_pipeline(X_processed, y)

with open('output/Phase_2/results_approach2.pkl', 'wb') as f:
    pickle.dump(results_l1, f)

print("\nAPPROACH 2 RESULTS SUMMARY")
print("Overall Performance:")
print(f"  Accuracy:     {np.mean(results_l1['accuracy']):.3f} ± {np.std(results_l1['accuracy']):.3f}")
print(f"  ROC-AUC:      {np.mean(results_l1['roc_auc']):.3f} ± {np.std(results_l1['roc_auc']):.3f}")
print(f"  MCC:          {np.mean(results_l1['mcc']):.3f} ± {np.std(results_l1['mcc']):.3f}")
print(f"  Features:     {np.mean(results_l1['n_features']):.0f} ± {np.std(results_l1['n_features']):.0f}")

print("\nPer-Fold Breakdown:")
for fold_idx in range(len(results_l1['accuracy'])):
    print(f"  Fold {fold_idx+1}: Acc={results_l1['accuracy'][fold_idx]:.3f} | "
          f"AUC={results_l1['roc_auc'][fold_idx]:.3f} | "
          f"MCC={results_l1['mcc'][fold_idx]:.3f} | "
          f"Features={results_l1['n_features'][fold_idx]}")

print("\nFeature Selection Analysis:")
print(f"  Min features:  {min(results_l1['n_features'])}")
print(f"  Max features:  {max(results_l1['n_features'])}")
print(f"  Median:        {np.median(results_l1['n_features']):.0f}")
print(f"  Consistency:   {np.std(results_l1['n_features'])/np.mean(results_l1['n_features']):.2%} (CV)")

print("\nPerformance Interpretation:")
mean_auc = np.mean(results_l1['roc_auc'])
if mean_auc >= 0.8:
    print("  EXCELLENT performance (AUC >= 0.8)")
elif mean_auc >= 0.7:
    print("  GOOD performance (0.7 <= AUC < 0.8)")
elif mean_auc >= 0.6:
    print("  FAIR performance (0.6 <= AUC < 0.7)")
else:
    print("  POOR performance (AUC < 0.6)")

approach2_detailed = pd.DataFrame({
    'Fold': range(1, len(results_l1['accuracy']) + 1),
    'Accuracy': results_l1['accuracy'],
    'ROC_AUC': results_l1['roc_auc'],
    'MCC': results_l1['mcc'],
    'Num_Features': results_l1['n_features']
})
approach2_detailed.to_csv('output/Phase_2/approach2_detailed.csv', index=False)

approach2_summary = pd.DataFrame({
    'Metric': ['Accuracy', 'ROC-AUC', 'MCC', 'Num_Features'],
    'Mean': [np.mean(results_l1['accuracy']), np.mean(results_l1['roc_auc']),
             np.mean(results_l1['mcc']), np.mean(results_l1['n_features'])],
    'Std': [np.std(results_l1['accuracy']), np.std(results_l1['roc_auc']),
            np.std(results_l1['mcc']), np.std(results_l1['n_features'])],
    'Min': [np.min(results_l1['accuracy']), np.min(results_l1['roc_auc']),
            np.min(results_l1['mcc']), min(results_l1['n_features'])],
    'Max': [np.max(results_l1['accuracy']), np.max(results_l1['roc_auc']),
            np.max(results_l1['mcc']), max(results_l1['n_features'])]
})
approach2_summary.to_csv('output/Phase_2/approach2_summary.csv', index=False)

print("\nResults saved:")
print("  output/Phase_2/results_approach2.pkl")
print("  output/Phase_2/approach2_detailed.csv")
print("  output/Phase_2/approach2_summary.csv")

Using data from memory

APPROACH 2: L1 (Lasso) Feature Selection + RBF-SVM
Dataset: 42 samples × 36157 genes
Method: LinearSVC L1 penalty + Nested CV

APPROACH 2: L1 (Lasso) Feature Selection + Non-linear Classifier


  Outer Folds:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits

  Mean Accuracy: 0.578 ± 0.118
  Mean ROC-AUC: 0.555 ± 0.166
  Mean MCC: 0.243 ± 0.223
  Mean Features: 7256 ± 14451

APPROACH 2 RESULTS SUMMARY
Overall Performance:
  Accuracy:     0.578 ± 0.118
  ROC-AUC:      0.555 ± 0.166
  MCC:          0.243 ± 0.223
  Features:     7256 ± 14451

Per-Fold Breakdown:
  Fold 1: Acc=0.444 | AUC=0.450 | MCC=0.000 | Features=36157
  Fold 2: Acc=0.444 | AUC=0.450 | MCC=0.000 | Features=28
  Fold 3: Acc=0.750 | AUC=0.688 | MCC=0.577 | Features=34
  Fold 4: Acc=0.625 | AUC=0.812 | MCC=0.258 | Features=32
  Fold 5: Acc=0.625 | AUC=0.375 | MCC=0.378 | Features=29

Feature Selection Analysis:
  Min features:  28
  Max features:  36157
  Median:        32
  C

In [17]:
try:
    X_processed
    print("Using data from memory")
except NameError:
    print("Loading from disk...")
    with open('output/Phase_2/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
        X_processed = data['X_processed']
        y = data['y']
        gene_names_filtered = data['gene_names_filtered']

print("\nAPPROACH 3: Stability Selection + RBF-SVM")
print(f"Dataset: {X_processed.shape[0]} samples × {X_processed.shape[1]} genes")
print(f"Method: Bootstrap stability selection (50 iterations)")

results_stability, stable_gene_indices, gene_scores = stability_selection_pipeline(
    X_processed, y, n_bootstrap=50, threshold=0.6
)

with open('output/Phase_2/results_approach3.pkl', 'wb') as f:
    pickle.dump({
        'results': results_stability,
        'stable_gene_indices': stable_gene_indices,
        'gene_scores': gene_scores
    }, f)

print("\nAPPROACH 3 RESULTS SUMMARY")
print("Overall Performance:")
print(f"  Accuracy:     {np.mean(results_stability['accuracy']):.3f} ± {np.std(results_stability['accuracy']):.3f}")
print(f"  ROC-AUC:      {np.mean(results_stability['roc_auc']):.3f} ± {np.std(results_stability['roc_auc']):.3f}")
print(f"  MCC:          {np.mean(results_stability['mcc']):.3f} ± {np.std(results_stability['mcc']):.3f}")
print(f"  Stable Genes: {len(stable_gene_indices)}")

print("\nPer-Fold Breakdown:")
for fold_idx in range(len(results_stability['accuracy'])):
    print(f"  Fold {fold_idx+1}: Acc={results_stability['accuracy'][fold_idx]:.3f} | "
          f"AUC={results_stability['roc_auc'][fold_idx]:.3f} | "
          f"MCC={results_stability['mcc'][fold_idx]:.3f}")

print("\nStable Genes Analysis:")
print(f"  Total selected: {len(stable_gene_indices)}")
print(f"  Stability scores - Min: {gene_scores[stable_gene_indices].min():.3f}, Max: {gene_scores[stable_gene_indices].max():.3f}")

if gene_names_filtered and len(stable_gene_indices) > 0:
    max_idx = max(stable_gene_indices)
    if max_idx < len(gene_names_filtered):
        stable_genes_df = pd.DataFrame({
            'gene': [gene_names_filtered[i] for i in stable_gene_indices],
            'stability_score': gene_scores[stable_gene_indices]
        }).sort_values('stability_score', ascending=False)
        
        print("\n  Top 10 Most Stable Genes:")
        for idx, row in stable_genes_df.head(10).iterrows():
            print(f"    {row['gene']}: {row['stability_score']:.3f}")
        
        stable_genes_df.to_csv('output/Phase_2/approach3_stable_genes.csv', index=False)
        print("  Full gene list saved to 'output/Phase_2/approach3_stable_genes.csv'")

print("\nPerformance Interpretation:")
mean_auc = np.mean(results_stability['roc_auc'])
if mean_auc >= 0.8:
    print("  EXCELLENT performance (AUC >= 0.8)")
elif mean_auc >= 0.7:
    print("  GOOD performance (0.7 <= AUC < 0.8)")
elif mean_auc >= 0.6:
    print("  FAIR performance (0.6 <= AUC < 0.7)")
else:
    print("  POOR performance (AUC < 0.6)")

approach3_detailed = pd.DataFrame({
    'Fold': range(1, len(results_stability['accuracy']) + 1),
    'Accuracy': results_stability['accuracy'],
    'ROC_AUC': results_stability['roc_auc'],
    'MCC': results_stability['mcc']
})
approach3_detailed.to_csv('output/Phase_2/approach3_detailed.csv', index=False)

approach3_summary = pd.DataFrame({
    'Metric': ['Accuracy', 'ROC-AUC', 'MCC', 'Stable_Genes'],
    'Mean': [np.mean(results_stability['accuracy']), np.mean(results_stability['roc_auc']),
             np.mean(results_stability['mcc']), len(stable_gene_indices)],
    'Std': [np.std(results_stability['accuracy']), np.std(results_stability['roc_auc']),
            np.std(results_stability['mcc']), 0],
    'Min': [np.min(results_stability['accuracy']), np.min(results_stability['roc_auc']),
            np.min(results_stability['mcc']), len(stable_gene_indices)],
    'Max': [np.max(results_stability['accuracy']), np.max(results_stability['roc_auc']),
            np.max(results_stability['mcc']), len(stable_gene_indices)]
})
approach3_summary.to_csv('output/Phase_2/approach3_summary.csv', index=False)

print("\nResults saved:")
print("  output/Phase_2/results_approach3.pkl")
print("  output/Phase_2/approach3_detailed.csv")
print("  output/Phase_2/approach3_summary.csv")

Using data from memory

APPROACH 3: Stability Selection + RBF-SVM
Dataset: 42 samples × 36157 genes
Method: Bootstrap stability selection (50 iterations)

APPROACH 3: Stability Selection + RBF-SVM
  Running 50 bootstrap iterations for stability selection...


  StableGeneSelector (n=50):   0%|          | 0/50 [00:00<?, ?it/s]

  Selected 50 stable genes

  Evaluating classifier on 50 stable features...


  Evaluation Folds:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits

  Mean Accuracy: 0.778 ± 0.272
  Mean ROC-AUC: 0.200 ± 0.245
  Mean MCC: 0.600 ± 0.490

APPROACH 3 RESULTS SUMMARY
Overall Performance:
  Accuracy:     0.778 ± 0.272
  ROC-AUC:      0.200 ± 0.245
  MCC:          0.600 ± 0.490
  Stable Genes: 50

Per-Fold Breakdown:
  Fold 1: Acc=0.444 | AUC=0.000 | MCC=0.000
  Fold 2: Acc=0.444 | AUC=0.000 | MCC=0.000
  Fold 3: Acc=1.000 | AUC=0.500 | MCC=1.000
  Fold 4: Acc=1.000 | AUC=0.500 | MCC=1.000
  Fold 5: Acc=1.000 | AUC=0.000 | MCC=1.000

Stable Genes Analysis:
  Total selected: 50
  Stability scores - Min: 0.060, Max: 0.232

  Top 10 Most Stable Genes:
    LOC284948: 0.232
    NTRK1: 0.170
    PPAP2C: 0.154
    LOC648517: 0.138
    WDR63: 0.

In [20]:
try:
    X_processed
    print("Using data from memory")
except NameError:
    print("Loading from disk...")
    with open('output/Phase_2/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
        X_processed = data['X_processed']
        y = data['y']
        gene_names_filtered = data['gene_names_filtered']

print("\nAPPROACH 4: Mutual Information + RFE Pipeline")
print(f"Dataset: {X_processed.shape[0]} samples × {X_processed.shape[1]} genes")
print(f"Method: MI pre-filter → RFE fine-tuning")

# Check if results already exist, if so load them, otherwise train
results_file = 'output/Phase_2/results_approach4.pkl'
if os.path.exists(results_file):
    with open(results_file, 'rb') as f:
        data = pickle.load(f)
        results_mi_rfe = data['results']
        best_config = data['best_config']
else:
    results_mi_rfe, best_config = mutual_information_rfe_pipeline(X_processed, y)
    with open(results_file, 'wb') as f:
        pickle.dump({'results': results_mi_rfe, 'best_config': best_config}, f)

print("\nAPPROACH 4 RESULTS SUMMARY")
print("Overall Performance:")
print(f"  Accuracy:     {np.mean(results_mi_rfe['accuracy']):.3f} ± {np.std(results_mi_rfe['accuracy']):.3f}")
print(f"  ROC-AUC:      {np.mean(results_mi_rfe['roc_auc']):.3f} ± {np.std(results_mi_rfe['roc_auc']):.3f}")
print(f"  MCC:          {np.mean(results_mi_rfe['mcc']):.3f} ± {np.std(results_mi_rfe['mcc']):.3f}")

print("\nPer-Fold Breakdown:")
for fold_idx in range(len(results_mi_rfe['accuracy'])):
    print(f"  Fold {fold_idx+1}: Acc={results_mi_rfe['accuracy'][fold_idx]:.3f} | "
          f"AUC={results_mi_rfe['roc_auc'][fold_idx]:.3f} | "
          f"MCC={results_mi_rfe['mcc'][fold_idx]:.3f}")

print("\nBest Configuration:")
print(f"  MI features:  {best_config[0]}")
print(f"  RFE features: {best_config[1]}")
print(f"  Reduction:    {best_config[0]} → {best_config[1]} ({best_config[1]/best_config[0]:.1%})")

print("\nPerformance Interpretation:")
mean_auc = np.mean(results_mi_rfe['roc_auc'])
if mean_auc >= 0.8:
    print("  EXCELLENT performance (AUC >= 0.8)")
elif mean_auc >= 0.7:
    print("  GOOD performance (0.7 <= AUC < 0.8)")
elif mean_auc >= 0.6:
    print("  FAIR performance (0.6 <= AUC < 0.7)")
else:
    print("  POOR performance (AUC < 0.6)")

approach4_detailed = pd.DataFrame({
    'Fold': range(1, len(results_mi_rfe['accuracy']) + 1),
    'Accuracy': results_mi_rfe['accuracy'],
    'ROC_AUC': results_mi_rfe['roc_auc'],
    'MCC': results_mi_rfe['mcc']
})
approach4_detailed.to_csv('output/Phase_2/approach4_detailed.csv', index=False)

approach4_summary = pd.DataFrame({
    'Metric': ['Accuracy', 'ROC-AUC', 'MCC', 'MI_Features', 'RFE_Features'],
    'Mean': [np.mean(results_mi_rfe['accuracy']), np.mean(results_mi_rfe['roc_auc']),
             np.mean(results_mi_rfe['mcc']), best_config[0], best_config[1]],
    'Std': [np.std(results_mi_rfe['accuracy']), np.std(results_mi_rfe['roc_auc']),
            np.std(results_mi_rfe['mcc']), 0, 0],
    'Min': [np.min(results_mi_rfe['accuracy']), np.min(results_mi_rfe['roc_auc']),
            np.min(results_mi_rfe['mcc']), best_config[0], best_config[1]],
    'Max': [np.max(results_mi_rfe['accuracy']), np.max(results_mi_rfe['roc_auc']),
            np.max(results_mi_rfe['mcc']), best_config[0], best_config[1]]
})
approach4_summary.to_csv('output/Phase_2/approach4_summary.csv', index=False)

print("\nResults saved:")
print("  output/Phase_2/results_approach4.pkl")
print("  output/Phase_2/approach4_detailed.csv")
print("  output/Phase_2/approach4_summary.csv")


Using data from memory

APPROACH 4: Mutual Information + RFE Pipeline
Dataset: 42 samples × 36157 genes
Method: MI pre-filter → RFE fine-tuning

APPROACH 4 RESULTS SUMMARY
Overall Performance:
  Accuracy:     0.528 ± 0.155
  ROC-AUC:      0.542 ± 0.140
  MCC:          0.062 ± 0.320

Per-Fold Breakdown:
  Fold 1: Acc=0.333 | AUC=0.350 | MCC=-0.350
  Fold 2: Acc=0.556 | AUC=0.550 | MCC=0.158
  Fold 3: Acc=0.625 | AUC=0.625 | MCC=0.258
  Fold 4: Acc=0.375 | AUC=0.438 | MCC=-0.258
  Fold 5: Acc=0.750 | AUC=0.750 | MCC=0.500

Best Configuration:
  MI features:  500
  RFE features: 30
  Reduction:    500 → 30 (6.0%)

Performance Interpretation:
  POOR performance (AUC < 0.6)

Results saved:
  output/Phase_2/results_approach4.pkl
  output/Phase_2/approach4_detailed.csv
  output/Phase_2/approach4_summary.csv


In [21]:
try:
    X_processed
    print("Using data from memory")
except NameError:
    print("Loading from disk...")
    with open('output/Phase_2/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
        X_processed = data['X_processed']
        y = data['y']
        gene_names_filtered = data['gene_names_filtered']

print("\nAPPROACH 5: Feature Bagging Ensemble")
print(f"Dataset: {X_processed.shape[0]} samples × {X_processed.shape[1]} genes")
print(f"Method: 30 SVMs on random 10% feature subsets")

# Check if results already exist, if so load them, otherwise train
results_file = 'output/Phase_2/results_approach5.pkl'
if os.path.exists(results_file):
    with open(results_file, 'rb') as f:
        results_ensemble = pickle.load(f)
else:
    results_ensemble = ensemble_feature_bagging(X_processed, y, n_estimators=30, feature_fraction=0.1)
    with open(results_file, 'wb') as f:
        pickle.dump(results_ensemble, f)

print("\nAPPROACH 5 RESULTS SUMMARY")
features_per_model = int(X_processed.shape[1] * 0.1)
print("Overall Performance:")
print(f"  Accuracy:     {np.mean(results_ensemble['accuracy']):.3f} ± {np.std(results_ensemble['accuracy']):.3f}")
print(f"  ROC-AUC:      {np.mean(results_ensemble['roc_auc']):.3f} ± {np.std(results_ensemble['roc_auc']):.3f}")
print(f"  MCC:          {np.mean(results_ensemble['mcc']):.3f} ± {np.std(results_ensemble['mcc']):.3f}")
print(f"  Ensemble:     30 models × {features_per_model} features")

print("\nPer-Fold Breakdown:")
for fold_idx in range(len(results_ensemble['accuracy'])):
    print(f"  Fold {fold_idx+1}: Acc={results_ensemble['accuracy'][fold_idx]:.3f} | "
          f"AUC={results_ensemble['roc_auc'][fold_idx]:.3f} | "
          f"MCC={results_ensemble['mcc'][fold_idx]:.3f}")

print("\nEnsemble Configuration:")
print(f"  Number of models:     30")
print(f"  Features per model:   {features_per_model}")
print(f"  Total features used:  {X_processed.shape[1]}")
print(f"  Feature fraction:     10%")
print(f"  Aggregation method:   Mean probability")

print("\nPerformance Interpretation:")
mean_auc = np.mean(results_ensemble['roc_auc'])
if mean_auc >= 0.8:
    print("  EXCELLENT performance (AUC >= 0.8)")
elif mean_auc >= 0.7:
    print("  GOOD performance (0.7 <= AUC < 0.8)")
elif mean_auc >= 0.6:
    print("  FAIR performance (0.6 <= AUC < 0.7)")
else:
    print("  POOR performance (AUC < 0.6)")

approach5_detailed = pd.DataFrame({
    'Fold': range(1, len(results_ensemble['accuracy']) + 1),
    'Accuracy': results_ensemble['accuracy'],
    'ROC_AUC': results_ensemble['roc_auc'],
    'MCC': results_ensemble['mcc']
})
approach5_detailed.to_csv('output/Phase_2/approach5_detailed.csv', index=False)

approach5_summary = pd.DataFrame({
    'Metric': ['Accuracy', 'ROC-AUC', 'MCC', 'Ensemble_Size', 'Features_Per_Model'],
    'Mean': [np.mean(results_ensemble['accuracy']), np.mean(results_ensemble['roc_auc']),
             np.mean(results_ensemble['mcc']), 30, features_per_model],
    'Std': [np.std(results_ensemble['accuracy']), np.std(results_ensemble['roc_auc']),
            np.std(results_ensemble['mcc']), 0, 0],
    'Min': [np.min(results_ensemble['accuracy']), np.min(results_ensemble['roc_auc']),
            np.min(results_ensemble['mcc']), 30, features_per_model],
    'Max': [np.max(results_ensemble['accuracy']), np.max(results_ensemble['roc_auc']),
            np.max(results_ensemble['mcc']), 30, features_per_model]
})
approach5_summary.to_csv('output/Phase_2/approach5_summary.csv', index=False)

print("\nResults saved:")
print("  output/Phase_2/results_approach5.pkl")
print("  output/Phase_2/approach5_detailed.csv")
print("  output/Phase_2/approach5_summary.csv")


Using data from memory

APPROACH 5: Feature Bagging Ensemble
Dataset: 42 samples × 36157 genes
Method: 30 SVMs on random 10% feature subsets

APPROACH 5 RESULTS SUMMARY
Overall Performance:
  Accuracy:     0.453 ± 0.046
  ROC-AUC:      0.532 ± 0.188
  MCC:          -0.107 ± 0.149
  Ensemble:     30 models × 3615 features

Per-Fold Breakdown:
  Fold 1: Acc=0.444 | AUC=0.450 | MCC=-0.158
  Fold 2: Acc=0.444 | AUC=0.900 | MCC=0.000
  Fold 3: Acc=0.500 | AUC=0.500 | MCC=0.000
  Fold 4: Acc=0.500 | AUC=0.438 | MCC=0.000
  Fold 5: Acc=0.375 | AUC=0.375 | MCC=-0.378

Ensemble Configuration:
  Number of models:     30
  Features per model:   3615
  Total features used:  36157
  Feature fraction:     10%
  Aggregation method:   Mean probability

Performance Interpretation:
  POOR performance (AUC < 0.6)

Results saved:
  output/Phase_2/results_approach5.pkl
  output/Phase_2/approach5_detailed.csv
  output/Phase_2/approach5_summary.csv


In [22]:
try:
    X_processed
    print("Using data from memory")
except NameError:
    print("Loading from disk...")
    with open('output/Phase_2/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
        X_processed = data['X_processed']
        y = data['y']
        gene_names_filtered = data['gene_names_filtered']

print("\nAPPROACH 6: OPTIMAL Pipeline (LOO-CV)")
print(f"Dataset: {X_processed.shape[0]} samples × {X_processed.shape[1]} genes")
print(f"Method: Leave-One-Out CV with nested stability selection")
print("WARNING: This will take longer but provides most accurate estimates")

# Check if results already exist, if so load them, otherwise train
results_file = 'output/Phase_2/results_optimal.pkl'
if os.path.exists(results_file):
    with open(results_file, 'rb') as f:
        results_optimal = pickle.load(f)
else:
    results_optimal = optimal_nested_cv_pipeline(X_processed, y, n_genes=50)
    with open(results_file, 'wb') as f:
        pickle.dump(results_optimal, f)

print("\nOPTIMAL PIPELINE RESULTS SUMMARY")
metrics = results_optimal['metrics']
cm = metrics['confusion_matrix']

print("Overall Performance:")
print(f"  Accuracy:     {metrics['accuracy']:.3f}")
print(f"  ROC-AUC:      {metrics['auc']:.3f}")
print(f"  MCC:          {metrics['mcc']:.3f}")

tn, fp, fn, tp = cm[0,0], cm[0,1], cm[1,0], cm[1,1]
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
f1_score = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
npv = tn / (tn + fn) if (tn + fn) > 0 else 0

print(f"  Sensitivity:  {sensitivity:.3f} (Recall, TPR)")
print(f"  Specificity:  {specificity:.3f} (TNR)")
print(f"  Precision:    {precision:.3f} (PPV)")
print(f"  F1-Score:     {f1_score:.3f}")
print(f"  NPV:          {npv:.3f}")

print("\nConfusion Matrix:")
print(f"                 Predicted Control  Predicted MDD")
print(f"  True Control          {tn:3d}              {fp:3d}")
print(f"  True MDD              {fn:3d}              {tp:3d}")
print(f"\n  True Negatives  (TN): {tn}")
print(f"  False Positives (FP): {fp}")
print(f"  False Negatives (FN): {fn}")
print(f"  True Positives  (TP): {tp}")

print("\nSample Predictions (First 10):")
predictions = np.array(results_optimal['predictions'])
probabilities = np.array(results_optimal['probabilities'])
true_labels = np.array(results_optimal['true_labels'])

for i in range(min(10, len(predictions))):
    status = "CORRECT" if predictions[i] == true_labels[i] else "WRONG"
    print(f"  Sample {i+1:2d}: True={true_labels[i]} | Pred={predictions[i]} | Prob={probabilities[i]:.3f} {status}")

print("\nGene Selection Consistency:")
gene_selection_freq = metrics['gene_selection_frequency']
consistent_genes = metrics['consistent_genes']

print(f"  Genes selected >50% of folds: {len(consistent_genes)}")
print(f"  Genes selected >75% of folds: {np.sum(gene_selection_freq > 0.75)}")
print(f"  Genes selected 100% of folds:  {np.sum(gene_selection_freq == 1.0)}")

if len(consistent_genes) > 0 and gene_names_filtered:
    top_consistent_indices = np.argsort(gene_selection_freq)[-20:][::-1]
    valid_indices = [i for i in top_consistent_indices if i < len(gene_names_filtered)]
    
    if valid_indices:
        print("\n  Top 10 Most Consistently Selected Genes:")
        for idx in valid_indices[:10]:
            print(f"    {gene_names_filtered[idx]}: {gene_selection_freq[idx]:.1%}")

print("\nPerformance Interpretation:")
if metrics['auc'] >= 0.8:
    print("  EXCELLENT performance (AUC >= 0.8)")
elif metrics['auc'] >= 0.7:
    print("  GOOD performance (0.7 <= AUC < 0.8)")
elif metrics['auc'] >= 0.6:
    print("  FAIR performance (0.6 <= AUC < 0.7)")
else:
    print("  POOR performance (AUC < 0.6)")

if metrics['mcc'] >= 0.5:
    print("  Strong correlation between predictions and reality")
elif metrics['mcc'] >= 0.3:
    print("  Moderate correlation")
elif metrics['mcc'] >= 0.1:
    print("  Weak correlation")
else:
    print("  No better than random guessing")

optimal_predictions = pd.DataFrame({
    'Sample': range(1, len(predictions) + 1),
    'True_Label': true_labels,
    'Predicted_Label': predictions,
    'Probability_MDD': probabilities,
    'Correct': predictions == true_labels
})
optimal_predictions.to_csv('output/Phase_2/optimal_predictions.csv', index=False)

cm_df = pd.DataFrame(cm, index=['True_Control', 'True_MDD'], columns=['Pred_Control', 'Pred_MDD'])
cm_df.to_csv('output/Phase_2/optimal_confusion_matrix.csv')

if gene_names_filtered:
    valid_mask = np.arange(len(gene_selection_freq)) < len(gene_names_filtered)
    gene_consistency = pd.DataFrame({
        'gene': [gene_names_filtered[i] for i in range(len(gene_names_filtered)) if valid_mask[i]],
        'selection_frequency': gene_selection_freq[valid_mask]
    }).sort_values('selection_frequency', ascending=False)
    gene_consistency.to_csv('output/Phase_2/optimal_gene_consistency.csv', index=False)

optimal_summary = pd.DataFrame({
    'Metric': ['Accuracy', 'ROC-AUC', 'MCC', 'Sensitivity', 'Specificity', 
               'Precision', 'F1-Score', 'NPV', 'Consistent_Genes'],
    'Value': [metrics['accuracy'], metrics['auc'], metrics['mcc'], sensitivity, 
              specificity, precision, f1_score, npv, len(consistent_genes)]
})
optimal_summary.to_csv('output/Phase_2/optimal_summary.csv', index=False)

print("\nResults saved:")
print("  output/Phase_2/results_optimal.pkl")
print("  output/Phase_2/optimal_predictions.csv")
print("  output/Phase_2/optimal_confusion_matrix.csv")
print("  output/Phase_2/optimal_gene_consistency.csv")
print("  output/Phase_2/optimal_summary.csv")

Using data from memory

APPROACH 6: OPTIMAL Pipeline (LOO-CV)
Dataset: 42 samples × 36157 genes
Method: Leave-One-Out CV with nested stability selection

OPTIMAL PIPELINE RESULTS SUMMARY
Overall Performance:
  Accuracy:     0.476
  ROC-AUC:      0.492
  MCC:          -0.048
  Sensitivity:  0.476 (Recall, TPR)
  Specificity:  0.476 (TNR)
  Precision:    0.476 (PPV)
  F1-Score:     0.476
  NPV:          0.476

Confusion Matrix:
                 Predicted Control  Predicted MDD
  True Control           10               11
  True MDD               11               10

  True Negatives  (TN): 10
  False Positives (FP): 11
  False Negatives (FN): 11
  True Positives  (TP): 10

Sample Predictions (First 10):
  Sample  1: True=1 | Pred=1 | Prob=0.635 CORRECT
  Sample  2: True=1 | Pred=0 | Prob=0.199 WRONG
  Sample  3: True=1 | Pred=1 | Prob=0.926 CORRECT
  Sample  4: True=1 | Pred=1 | Prob=0.783 CORRECT
  Sample  5: True=1 | Pred=1 | Prob=0.957 CORRECT
  Sample  6: True=1 | Pred=1 | Prob=0.973

In [23]:
import os

try:
    X_processed
    print("Using data from memory")
except NameError:
    print("Loading from disk...")
    with open('output/Phase_2/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
        X_processed = data['X_processed']
        y = data['y']
        gene_names_filtered = data['gene_names_filtered']

print("\nFINAL MODEL TRAINING FOR DEPLOYMENT")
print(f"Dataset: {X_processed.shape[0]} samples × {X_processed.shape[1]} genes")
print(f"Training on 100% of data for production use")

print("\nPre-Training Verification:")
print(f"  X_processed shape:  {X_processed.shape}")
print(f"  y shape:            {y.shape}")
print(f"  gene_names length:  {len(gene_names_filtered) if gene_names_filtered else None}")

if gene_names_filtered and len(gene_names_filtered) != X_processed.shape[1]:
    print(f"  WARNING: Mismatch detected!")
    print(f"     Features: {X_processed.shape[1]}")
    print(f"     Gene names: {len(gene_names_filtered)}")
    print(f"  Truncating gene_names to match features")
    gene_names_filtered = gene_names_filtered[:X_processed.shape[1]]

print("\nTraining final model...")

# Check if final model already exists, if so load it, otherwise train
model_file = 'output/Phase_2/final_model.pkl'
if os.path.exists(model_file):
    with open(model_file, 'rb') as f:
        final_pipeline = pickle.load(f)
    
    selector = final_pipeline['selector']
    scaler = final_pipeline['scaler']
    final_clf = final_pipeline['classifier']
    selected_indices = final_pipeline['selected_indices']
    train_acc = final_pipeline['training_performance']['accuracy']
    train_auc = final_pipeline['training_performance']['roc_auc']
    train_mcc = final_pipeline['training_performance']['mcc']
    selected_gene_names = final_pipeline.get('selected_genes', None)
    gene_importance = final_pipeline.get('gene_importance', None)
    X_selected = selector.transform(X_processed)
else:
    selector = StableGeneSelector(n_genes_to_select=min(50, X_processed.shape[1]),
                                 n_bootstrap=50, stability_threshold=0.5, random_state=42)
    X_selected = selector.fit_transform(X_processed, y)
    print(f"Feature selection complete: {X_selected.shape[1]} genes selected")

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    print("Scaling complete")

    final_clf = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42)
    final_clf.fit(X_scaled, y)
    print("Classifier training complete")

    print("\nTraining Set Performance:")
    y_pred_train = final_clf.predict(X_scaled)
    y_prob_train = final_clf.predict_proba(X_scaled)[:, 1]

    train_acc = accuracy_score(y, y_pred_train)
    train_auc = roc_auc_score(y, y_prob_train)
    train_mcc = matthews_corrcoef(y, y_pred_train)
    
    selected_indices = selector.selected_genes_
    
    if gene_names_filtered:
        max_idx = max(selected_indices)
        if max_idx >= len(gene_names_filtered):
            selected_gene_names = None
            gene_importance = None
        else:
            selected_gene_names = [gene_names_filtered[i] for i in selected_indices]
            gene_importance = pd.DataFrame({
                'gene': selected_gene_names,
                'stability_score': selector.gene_scores_[selected_indices]
            }).sort_values('stability_score', ascending=False)
            gene_importance['rank'] = range(1, len(gene_importance) + 1)
            gene_importance = gene_importance[['rank', 'gene', 'stability_score']]
    else:
        selected_gene_names = None
        gene_importance = None
    
    final_pipeline = {
        'selector': selector,
        'scaler': scaler,
        'classifier': final_clf,
        'selected_indices': selected_indices,
        'training_performance': {'accuracy': train_acc, 'roc_auc': train_auc, 'mcc': train_mcc},
        'metadata': {
            'n_training_samples': X_processed.shape[0],
            'n_original_features': X_processed.shape[1],
            'n_selected_features': X_selected.shape[1],
            'feature_selection_method': 'StableGeneSelector',
            'n_bootstrap': 50,
            'stability_threshold': 0.5,
            'classifier': 'RBF-SVM',
            'hyperparameters': {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'}
        }
    }
    
    if selected_gene_names:
        final_pipeline['selected_genes'] = selected_gene_names
        final_pipeline['gene_importance'] = gene_importance
    
    with open(model_file, 'wb') as f:
        pickle.dump(final_pipeline, f)

print(f"Feature selection complete: {X_selected.shape[1]} genes selected")
print("Scaling complete")
print("Classifier training complete")

print("\nTraining Set Performance:")
print(f"  Accuracy:  {train_acc:.3f}")
print(f"  ROC-AUC:   {train_auc:.3f}")
print(f"  MCC:       {train_mcc:.3f}")
print("  Note: These are optimistic estimates (trained on same data)")

print("\nSelected Gene Analysis:")

print(f"  Number of genes selected: {len(selected_indices)}")
print(f"  Index range: {min(selected_indices)} to {max(selected_indices)}")

if gene_names_filtered:
    print(f"  Gene names available: {len(gene_names_filtered)}")
    
    max_idx = max(selected_indices)
    if max_idx >= len(gene_names_filtered):
        print(f"  ERROR: Max index {max_idx} exceeds gene_names length {len(gene_names_filtered)}")
        print(f"  Cannot create gene importance table")
    else:
        print(f"\n  Top 20 Most Important Genes:")
        for _, row in gene_importance.head(20).iterrows():
            print(f"    {row['rank']:2d}. {row['gene']:20s} (score: {row['stability_score']:.3f})")
        
        gene_importance.to_csv('output/Phase_2/final_model_gene_importance.csv', index=False)
        print(f"  Full gene list saved to 'output/Phase_2/final_model_gene_importance.csv'")
else:
    print("  No gene names available - saving indices only")

print("\nPackaging Final Model:")
print("  Model components: Selector, Scaler, Classifier, Gene importance, Metadata")

metadata_df = pd.DataFrame({
    'Parameter': ['Training Samples', 'Original Features', 'Selected Features', 
                  'Feature Selection Method', 'Bootstrap Iterations', 'Stability Threshold',
                  'Classifier Type', 'SVM Kernel', 'SVM C', 'SVM Gamma',
                  'Training Accuracy', 'Training ROC-AUC', 'Training MCC'],
    'Value': [final_pipeline['metadata']['n_training_samples'],
              final_pipeline['metadata']['n_original_features'],
              final_pipeline['metadata']['n_selected_features'],
              final_pipeline['metadata']['feature_selection_method'],
              final_pipeline['metadata']['n_bootstrap'],
              final_pipeline['metadata']['stability_threshold'],
              final_pipeline['metadata']['classifier'],
              final_pipeline['metadata']['hyperparameters']['kernel'],
              final_pipeline['metadata']['hyperparameters']['C'],
              final_pipeline['metadata']['hyperparameters']['gamma'],
              f"{train_acc:.3f}", f"{train_auc:.3f}", f"{train_mcc:.3f}"]
})
metadata_df.to_csv('output/Phase_2/final_model_metadata.csv', index=False)

print("\nFinal model saved:")
print("  output/Phase_2/final_model.pkl")
print("  output/Phase_2/final_model_gene_importance.csv")
print("  output/Phase_2/final_model_metadata.csv")

Using data from memory

FINAL MODEL TRAINING FOR DEPLOYMENT
Dataset: 42 samples × 36157 genes
Training on 100% of data for production use

Pre-Training Verification:
  X_processed shape:  (42, 36157)
  y shape:            (42,)
  gene_names length:  36157

Training final model...
Feature selection complete: 50 genes selected
Scaling complete
Classifier training complete

Training Set Performance:
  Accuracy:  1.000
  ROC-AUC:   1.000
  MCC:       1.000
  Note: These are optimistic estimates (trained on same data)

Selected Gene Analysis:
  Number of genes selected: 50
  Index range: 240 to 36149
  Gene names available: 36157

  Top 20 Most Important Genes:
     1. LOC284948            (score: 0.232)
     2. NTRK1                (score: 0.170)
     3. PPAP2C               (score: 0.154)
     4. LOC648517            (score: 0.138)
     5. WDR63                (score: 0.122)
     6. WTAP                 (score: 0.114)
     7. LOC653513            (score: 0.110)
     8. POGZ               

In [24]:
import os
import glob

print("\nCOMPREHENSIVE RESULTS SUMMARY")

print("\nChecking saved results...")

available_results = {}

for i in range(1, 6):
    pkl_path = f'output/Phase_2/results_approach{i}.pkl'
    if os.path.exists(pkl_path):
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            if isinstance(data, dict) and 'results' in data:
                available_results[f'Approach {i}'] = data['results']
            else:
                available_results[f'Approach {i}'] = data
        print(f"  Approach {i} results loaded")
    else:
        print(f"  Approach {i} not found")

if os.path.exists('output/Phase_2/results_optimal.pkl'):
    with open('output/Phase_2/results_optimal.pkl', 'rb') as f:
        optimal_data = pickle.load(f)
        available_results['Optimal (LOO-CV)'] = optimal_data
    print(f"  Optimal pipeline results loaded")

if os.path.exists('output/Phase_2/final_model.pkl'):
    with open('output/Phase_2/final_model.pkl', 'rb') as f:
        final_model = pickle.load(f)
    print(f"  Final model loaded")

print("\nPERFORMANCE COMPARISON:")

comparison_data = []

for name, results in available_results.items():
    if name == 'Optimal (LOO-CV)':
        metrics = results['metrics']
        comparison_data.append({
            'Approach': name,
            'Accuracy': f"{metrics['accuracy']:.3f}",
            'ROC-AUC': f"{metrics['auc']:.3f}",
            'MCC': f"{metrics['mcc']:.3f}",
            'Rank': 0
        })
    else:
        comparison_data.append({
            'Approach': name,
            'Accuracy': f"{np.mean(results['accuracy']):.3f} ± {np.std(results['accuracy']):.3f}",
            'ROC-AUC': f"{np.mean(results['roc_auc']):.3f} ± {np.std(results['roc_auc']):.3f}",
            'MCC': f"{np.mean(results['mcc']):.3f} ± {np.std(results['mcc']):.3f}",
            'Rank': 0
        })

comparison_df = pd.DataFrame(comparison_data)

auc_values = []
for auc_str in comparison_df['ROC-AUC']:
    auc_val = float(auc_str.split('±')[0].strip())
    auc_values.append(auc_val)

comparison_df['Rank'] = pd.Series(auc_values).rank(ascending=False, method='min').astype(int)
comparison_df = comparison_df.sort_values('Rank')

print(comparison_df.to_string(index=False))

comparison_df.to_csv('output/Phase_2/all_approaches_comparison.csv', index=False)

print("\nBEST PERFORMING APPROACH:")

best_approach = comparison_df.iloc[0]['Approach']
best_auc = comparison_df.iloc[0]['ROC-AUC']

print(f"  {best_approach}")
print(f"  ROC-AUC: {best_auc}")

print("\nDETAILED METRICS BREAKDOWN:")

all_metrics = {'Accuracy': [], 'ROC-AUC': [], 'MCC': []}
approach_names = []

for name, results in available_results.items():
    approach_names.append(name)
    
    if name == 'Optimal (LOO-CV)':
        metrics = results['metrics']
        all_metrics['Accuracy'].append(metrics['accuracy'])
        all_metrics['ROC-AUC'].append(metrics['auc'])
        all_metrics['MCC'].append(metrics['mcc'])
    else:
        all_metrics['Accuracy'].append(np.mean(results['accuracy']))
        all_metrics['ROC-AUC'].append(np.mean(results['roc_auc']))
        all_metrics['MCC'].append(np.mean(results['mcc']))

for metric_name, values in all_metrics.items():
    if values:
        best_idx = np.argmax(values)
        worst_idx = np.argmin(values)
        
        print(f"\n{metric_name}:")
        print(f"  Best:  {approach_names[best_idx]:20s} = {values[best_idx]:.3f}")
        print(f"  Worst: {approach_names[worst_idx]:20s} = {values[worst_idx]:.3f}")
        print(f"  Range: {max(values) - min(values):.3f}")

print("\nSAVED FILES INVENTORY:")

all_files = glob.glob('output/Phase_2/*.pkl') + glob.glob('output/Phase_2/*.csv')
all_files.sort()

total_size = 0
for filepath in all_files:
    size_kb = os.path.getsize(filepath) / 1024
    total_size += size_kb
    print(f"  {os.path.basename(filepath):40s} {size_kb:8.1f} KB")

print(f"\n  Total: {len(all_files)} files, {total_size:.1f} KB")

print("\nRECOMMENDATIONS:")

best_auc_value = max(all_metrics['ROC-AUC'])

if best_auc_value >= 0.8:
    print("  Excellent performance achieved")
    print("  Recommended: Deploy the final model")
    print("  Consider: External validation on independent dataset")
elif best_auc_value >= 0.7:
    print("  Good performance achieved")
    print("  Recommended: Use for preliminary screening")
    print("  Consider: Collecting more samples or additional features")
elif best_auc_value >= 0.6:
    print("  Fair performance")
    print("  Recommended: Further feature engineering or try different algorithms")
    print("  Consider: Investigating biological heterogeneity in the cohort")
else:
    print("  Poor performance")
    print("  Recommended: Re-evaluate approach")
    print("  Consider: Different feature types, larger sample size, or problem reformulation")

print("\nComprehensive summary complete")
print("All results saved to 'output/Phase_2/all_approaches_comparison.csv'")


COMPREHENSIVE RESULTS SUMMARY

Checking saved results...
  Approach 1 results loaded
  Approach 2 results loaded
  Approach 3 results loaded
  Approach 4 results loaded
  Approach 5 results loaded
  Optimal pipeline results loaded
  Final model loaded

PERFORMANCE COMPARISON:
        Approach      Accuracy       ROC-AUC            MCC  Rank
      Approach 1 0.525 ± 0.061 0.685 ± 0.074  0.072 ± 0.101     1
      Approach 2 0.578 ± 0.118 0.555 ± 0.166  0.243 ± 0.223     2
      Approach 4 0.528 ± 0.155 0.542 ± 0.140  0.062 ± 0.320     3
      Approach 5 0.453 ± 0.046 0.532 ± 0.188 -0.107 ± 0.149     4
Optimal (LOO-CV)         0.476         0.492         -0.048     5
      Approach 3 0.778 ± 0.272 0.200 ± 0.245  0.600 ± 0.490     6

BEST PERFORMING APPROACH:
  Approach 1
  ROC-AUC: 0.685 ± 0.074

DETAILED METRICS BREAKDOWN:

Accuracy:
  Best:  Approach 3           = 0.778
  Worst: Approach 5           = 0.453
  Range: 0.325

ROC-AUC:
  Best:  Approach 1           = 0.685
  Worst: Approac

In [25]:
def predict_new_samples(pipeline, X_new):
    """
    Make predictions on new samples using the trained pipeline.
    
    Parameters:
    -----------
    pipeline : dict
        The final pipeline from train_final_model()
    X_new : array-like, shape (n_samples, n_genes)
        New samples to predict. Must be preprocessed the same way as training data.
    
    Returns:
    --------
    predictions : array
        Binary predictions (0=Control, 1=MDD)
    probabilities : array
        Probability of MDD class (0 to 1)
    confidence : array
        Prediction confidence (distance from 0.5 threshold)
    """
    
    X_selected = pipeline['selector'].transform(X_new)
    X_scaled = pipeline['scaler'].transform(X_selected)
    predictions = pipeline['classifier'].predict(X_scaled)
    probabilities = pipeline['classifier'].predict_proba(X_scaled)[:, 1]
    confidence = np.abs(probabilities - 0.5)
    
    return predictions, probabilities, confidence

print("\nTESTING PREDICTION FUNCTION")

try:
    final_pipeline
    print("Using final model from memory")
except NameError:
    print("Loading final model from disk...")
    with open('output/Phase_2/final_model.pkl', 'rb') as f:
        final_pipeline = pickle.load(f)

try:
    X_processed
    print("Using data from memory")
except NameError:
    print("Loading data from disk...")
    with open('output/Phase_2/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
        X_processed = data['X_processed']
        y = data['y']

print("\nMaking predictions on training data (sanity check):")

predictions, probabilities, confidence = predict_new_samples(final_pipeline, X_processed)

print("\nFirst 10 samples:")
print(f"{'Sample':<8} {'True':^6} {'Pred':^6} {'Probability':^12} {'Confidence':^10} {'Status':^8}")

for i in range(min(10, len(predictions))):
    status = "MATCH" if predictions[i] == y[i] else "MISS"
    label_true = "MDD" if y[i] == 1 else "Control"
    label_pred = "MDD" if predictions[i] == 1 else "Control"
    
    print(f"{i+1:<8} {label_true:^6} {label_pred:^6} {probabilities[i]:^12.3f} {confidence[i]:^10.3f} {status:^8}")

accuracy = np.mean(predictions == y)
print(f"\nTraining set accuracy: {accuracy:.3f}")
print("(Should be high since model was trained on this data)")

print("\nPrediction Confidence Distribution:")

low_conf = np.sum(confidence < 0.2)
med_conf = np.sum((confidence >= 0.2) & (confidence < 0.4))
high_conf = np.sum(confidence >= 0.4)

print(f"  High confidence (>0.4):      {high_conf:3d} samples ({high_conf/len(confidence):.1%})")
print(f"  Medium confidence (0.2-0.4): {med_conf:3d} samples ({med_conf/len(confidence):.1%})")
print(f"  Low confidence (<0.2):       {low_conf:3d} samples ({low_conf/len(confidence):.1%})")

print("\nSaving standalone prediction function...")

prediction_code = '''
import pickle
import numpy as np

def predict_mdd(X_new, model_path='output/Phase_2/final_model.pkl'):
    """
    Predict MDD status for new samples.
    
    Parameters:
    -----------
    X_new : array-like, shape (n_samples, n_genes)
        Preprocessed gene expression data (log2 + z-score normalized)
    model_path : str
        Path to saved model file
    
    Returns:
    --------
    predictions : array (0=Control, 1=MDD)
    probabilities : array (probability of MDD)
    confidence : array (prediction confidence)
    """
    
    with open(model_path, 'rb') as f:
        pipeline = pickle.load(f)
    
    X_selected = pipeline['selector'].transform(X_new)
    X_scaled = pipeline['scaler'].transform(X_selected)
    predictions = pipeline['classifier'].predict(X_scaled)
    probabilities = pipeline['classifier'].predict_proba(X_scaled)[:, 1]
    confidence = np.abs(probabilities - 0.5)
    
    return predictions, probabilities, confidence
'''

with open('output/Phase_2/predict_mdd.py', 'w') as f:
    f.write(prediction_code)

print("Standalone prediction function saved to 'output/Phase_2/predict_mdd.py'")



TESTING PREDICTION FUNCTION
Using final model from memory
Using data from memory

Making predictions on training data (sanity check):

First 10 samples:
Sample    True   Pred  Probability  Confidence  Status 
1         MDD    MDD      0.991       0.491     MATCH  
2         MDD    MDD      0.991       0.491     MATCH  
3         MDD    MDD      0.991       0.491     MATCH  
4         MDD    MDD      0.991       0.491     MATCH  
5         MDD    MDD      0.991       0.491     MATCH  
6         MDD    MDD      0.991       0.491     MATCH  
7         MDD    MDD      0.991       0.491     MATCH  
8         MDD    MDD      0.991       0.491     MATCH  
9         MDD    MDD      0.991       0.491     MATCH  
10        MDD    MDD      0.991       0.491     MATCH  

Training set accuracy: 1.000
(Should be high since model was trained on this data)

Prediction Confidence Distribution:
  High confidence (>0.4):       42 samples (100.0%)
  Medium confidence (0.2-0.4):   0 samples (0.0%)
  Low c

In [32]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['figure.titlesize'] = 14

print("\n" + "=" * 80)
print("CREATING VISUALIZATIONS")
print("=" * 80)

print("\nLoading all results...")

approach_results = {}
for i in range(1, 6):
    try:
        with open(f'output/Phase_2/results_approach{i}.pkl', 'rb') as f:
            data = pickle.load(f)
            if isinstance(data, dict) and 'results' in data:
                approach_results[f'Approach {i}'] = data['results']
            else:
                approach_results[f'Approach {i}'] = data
        print(f"  Approach {i} loaded")
    except:
        print(f"  Approach {i} not found")

try:
    with open('output/Phase_2/results_optimal.pkl', 'rb') as f:
        optimal_results = pickle.load(f)
    print(f"  Optimal pipeline loaded")
except:
    print(f"  Optimal pipeline not found")
    optimal_results = None

try:
    with open('output/Phase_2/final_model.pkl', 'rb') as f:
        final_model = pickle.load(f)
    print(f"  Final model loaded")
except:
    print(f"  Final model not found")
    final_model = None

print("\nAll available data loaded\n")

print("Creating Figure 1: Overall Performance Comparison...")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Performance Comparison Across All Approaches', fontsize=16, fontweight='bold')

metrics = ['accuracy', 'roc_auc', 'mcc']
titles = ['Accuracy', 'ROC-AUC', 'MCC (Matthews Correlation)']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for idx, (metric, title, color) in enumerate(zip(metrics, titles, colors)):
    ax = axes[idx]
    
    approach_names = []
    means = []
    stds = []
    
    for name, results in approach_results.items():
        approach_names.append(name.replace('Approach ', 'A'))
        means.append(np.mean(results[metric]))
        stds.append(np.std(results[metric]))
    
    if optimal_results:
        approach_names.append('Optimal')
        if metric == 'accuracy':
            means.append(optimal_results['metrics']['accuracy'])
        elif metric == 'roc_auc':
            means.append(optimal_results['metrics']['auc'])
        elif metric == 'mcc':
            means.append(optimal_results['metrics']['mcc'])
        stds.append(0)
    
    x_pos = np.arange(len(approach_names))
    bars = ax.bar(x_pos, means, yerr=stds, capsize=5, 
                   color=color, alpha=0.7, edgecolor='black', linewidth=1.5)
    
    best_idx = np.argmax(means)
    bars[best_idx].set_alpha(1.0)
    bars[best_idx].set_edgecolor('gold')
    bars[best_idx].set_linewidth(3)
    
    ax.set_xlabel('Approach', fontweight='bold')
    ax.set_ylabel(title, fontweight='bold')
    ax.set_title(title, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(approach_names, rotation=45, ha='right')
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim([0, 1.0])
    
    if metric in ['accuracy', 'roc_auc']:
        ax.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, linewidth=1, label='Random')
    if metric == 'roc_auc':
        ax.axhline(y=0.7, color='orange', linestyle='--', alpha=0.5, linewidth=1, label='Good')
        ax.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, linewidth=1, label='Excellent')
    
    for i, (m, s) in enumerate(zip(means, stds)):
        ax.text(i, m + s + 0.02, f'{m:.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('output/Phase_2/figure1_performance_comparison.png', dpi=300, bbox_inches='tight')
print("  Saved: output/Phase_2/figure1_performance_comparison.png")
plt.close()

print("Creating Figure 2: Performance Variability Across Folds...")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Performance Variability Across Cross-Validation Folds', fontsize=16, fontweight='bold')

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[idx]
    
    data_for_box = []
    labels_for_box = []
    
    for name, results in approach_results.items():
        data_for_box.append(results[metric])
        labels_for_box.append(name.replace('Approach ', 'A'))
    
    bp = ax.boxplot(data_for_box, labels=labels_for_box, patch_artist=True,
                     showmeans=True, meanline=True)
    
    colors_box = plt.cm.Set3(np.linspace(0, 1, len(data_for_box)))
    for patch, color in zip(bp['boxes'], colors_box):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', linewidth=1.5)
    plt.setp(bp['means'], color='red', linewidth=2)
    plt.setp(bp['medians'], color='blue', linewidth=2)
    
    ax.set_xlabel('Approach', fontweight='bold')
    ax.set_ylabel(title, fontweight='bold')
    ax.set_title(f'{title} Distribution', fontweight='bold')
    ax.set_xticklabels(labels_for_box, rotation=45, ha='right')
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim([0, 1.0])

plt.tight_layout()
plt.savefig('output/Phase_2/figure2_performance_variability.png', dpi=300, bbox_inches='tight')
print("  Saved: output/Phase_2/figure2_performance_variability.png")
plt.close()

print("Creating Figure 3: ROC-AUC with Confidence Intervals...")

fig, ax = plt.subplots(figsize=(12, 6))
fig.suptitle('ROC-AUC Scores with 95% Confidence Intervals', fontsize=16, fontweight='bold')

approach_names = []
auc_means = []
auc_stds = []
auc_mins = []
auc_maxs = []

for name, results in approach_results.items():
    approach_names.append(name.replace('Approach ', 'A'))
    auc_data = results['roc_auc']
    auc_means.append(np.mean(auc_data))
    auc_stds.append(np.std(auc_data))
    auc_mins.append(np.min(auc_data))
    auc_maxs.append(np.max(auc_data))

if optimal_results:
    approach_names.append('Optimal')
    auc_means.append(optimal_results['metrics']['auc'])
    auc_stds.append(0)
    auc_mins.append(optimal_results['metrics']['auc'])
    auc_maxs.append(optimal_results['metrics']['auc'])

x_pos = np.arange(len(approach_names))

ax.plot(x_pos, auc_means, 'o-', linewidth=2, markersize=10, color='#e74c3c', label='Mean AUC')

ci_lower = np.array(auc_means) - 1.96 * np.array(auc_stds)
ci_upper = np.array(auc_means) + 1.96 * np.array(auc_stds)
ax.fill_between(x_pos, ci_lower, ci_upper, alpha=0.3, color='#e74c3c', label='95% CI')

ax.fill_between(x_pos, auc_mins, auc_maxs, alpha=0.1, color='gray', label='Min-Max Range')

ax.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, linewidth=2, label='Random (0.5)')
ax.axhline(y=0.7, color='orange', linestyle='--', alpha=0.5, linewidth=2, label='Good (0.7)')
ax.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, linewidth=2, label='Excellent (0.8)')

ax.set_xlabel('Approach', fontweight='bold', fontsize=12)
ax.set_ylabel('ROC-AUC Score', fontweight='bold', fontsize=12)
ax.set_xticks(x_pos)
ax.set_xticklabels(approach_names, rotation=45, ha='right')
ax.set_ylim([0, 1.0])
ax.grid(True, alpha=0.3)
ax.legend(loc='lower right', framealpha=0.9)

best_idx = np.argmax(auc_means)
ax.annotate(f'Best: {auc_means[best_idx]:.3f}', 
            xy=(best_idx, auc_means[best_idx]), 
            xytext=(best_idx, auc_means[best_idx] + 0.1),
            arrowprops=dict(arrowstyle='->', color='gold', lw=2),
            fontsize=11, fontweight='bold', color='gold',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='black', alpha=0.7))

plt.tight_layout()
plt.savefig('output/Phase_2/figure3_roc_auc_confidence.png', dpi=300, bbox_inches='tight')
print("  Saved: output/Phase_2/figure3_roc_auc_confidence.png")
plt.close()

if optimal_results:
    print("Creating Figure 4: Confusion Matrix Heatmap...")
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle('Optimal Pipeline: Confusion Matrix Analysis', fontsize=16, fontweight='bold')
    
    cm = optimal_results['metrics']['confusion_matrix']
    
    ax = axes[0]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, 
                square=True, ax=ax, linewidths=2, linecolor='black',
                annot_kws={'size': 16, 'weight': 'bold'})
    ax.set_xlabel('Predicted Label', fontweight='bold', fontsize=12)
    ax.set_ylabel('True Label', fontweight='bold', fontsize=12)
    ax.set_title('Confusion Matrix (Counts)', fontweight='bold')
    ax.set_xticklabels(['Control', 'MDD'], fontsize=11)
    ax.set_yticklabels(['Control', 'MDD'], fontsize=11, rotation=0)
    
    ax = axes[1]
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='RdYlGn', cbar=True,
                square=True, ax=ax, linewidths=2, linecolor='black',
                annot_kws={'size': 16, 'weight': 'bold'}, vmin=0, vmax=1)
    ax.set_xlabel('Predicted Label', fontweight='bold', fontsize=12)
    ax.set_ylabel('True Label', fontweight='bold', fontsize=12)
    ax.set_title('Confusion Matrix (Normalized)', fontweight='bold')
    ax.set_xticklabels(['Control', 'MDD'], fontsize=11)
    ax.set_yticklabels(['Control', 'MDD'], fontsize=11, rotation=0)
    
    plt.tight_layout()
    plt.savefig('output/Phase_2/figure4_confusion_matrix.png', dpi=300, bbox_inches='tight')
    print("  Saved: output/Phase_2/figure4_confusion_matrix.png")
    plt.close()

print("Creating Figure 5: Feature Selection Analysis...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Feature Selection Analysis Across Approaches', fontsize=16, fontweight='bold')

ax = axes[0, 0]
approach_names_feat = []
n_features = []

for name, results in approach_results.items():
    if 'n_features' in results:
        approach_names_feat.append(name.replace('Approach ', 'A'))
        n_features.append(np.mean(results['n_features']))

if len(n_features) > 0:
    bars = ax.barh(approach_names_feat, n_features, color='skyblue', edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Average Number of Features Selected', fontweight='bold')
    ax.set_ylabel('Approach', fontweight='bold')
    ax.set_title('A) Feature Selection Size', fontweight='bold', loc='left')
    ax.grid(True, alpha=0.3, axis='x')
    
    for i, v in enumerate(n_features):
        ax.text(v + max(n_features)*0.02, i, f'{v:.0f}', va='center', fontweight='bold')
else:
    ax.text(0.5, 0.5, 'Feature count data not available', 
            ha='center', va='center', transform=ax.transAxes, fontsize=12)
    ax.set_title('A) Feature Selection Size', fontweight='bold', loc='left')

ax = axes[0, 1]
if optimal_results and 'gene_selection_frequency' in optimal_results['metrics']:
    gene_freq = optimal_results['metrics']['gene_selection_frequency']
    
    ax.hist(gene_freq[gene_freq > 0], bins=50, color='coral', edgecolor='black', alpha=0.7)
    ax.set_xlabel('Gene Selection Frequency', fontweight='bold')
    ax.set_ylabel('Number of Genes', fontweight='bold')
    ax.set_title('B) Gene Selection Stability (Optimal)', fontweight='bold', loc='left')
    ax.grid(True, alpha=0.3, axis='y')
    
    consistent = np.sum(gene_freq > 0.5)
    ax.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label=f'50% threshold ({consistent} genes)')
    ax.legend()
else:
    ax.text(0.5, 0.5, 'Gene stability data not available', 
            ha='center', va='center', transform=ax.transAxes, fontsize=12)
    ax.set_title('B) Gene Selection Stability', fontweight='bold', loc='left')

ax = axes[1, 0]
if final_model and 'gene_importance' in final_model:
    gene_imp = final_model['gene_importance'].head(15)
    
    y_pos = np.arange(len(gene_imp))
    ax.barh(y_pos, gene_imp['stability_score'], color='lightgreen', edgecolor='black', linewidth=1.5)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(gene_imp['gene'], fontsize=8)
    ax.set_xlabel('Stability Score', fontweight='bold')
    ax.set_ylabel('Gene', fontweight='bold')
    ax.set_title('C) Top 15 Most Stable Genes', fontweight='bold', loc='left')
    ax.grid(True, alpha=0.3, axis='x')
    ax.invert_yaxis()
else:
    ax.text(0.5, 0.5, 'Gene importance data not available', 
            ha='center', va='center', transform=ax.transAxes, fontsize=12)
    ax.set_title('C) Top Genes', fontweight='bold', loc='left')

ax = axes[1, 1]
perf_vs_feat = []

for name, results in approach_results.items():
    if 'n_features' in results:
        mean_auc = np.mean(results['roc_auc'])
        mean_features = np.mean(results['n_features'])
        perf_vs_feat.append({
            'approach': name.replace('Approach ', 'A'),
            'auc': mean_auc,
            'features': mean_features
        })

if len(perf_vs_feat) > 0:
    df_pf = pd.DataFrame(perf_vs_feat)
    
    scatter = ax.scatter(df_pf['features'], df_pf['auc'], s=200, c=df_pf['auc'], 
                         cmap='RdYlGn', edgecolors='black', linewidth=2, alpha=0.8)
    
    for _, row in df_pf.iterrows():
        ax.annotate(row['approach'], (row['features'], row['auc']), 
                    fontsize=9, ha='center', va='bottom', fontweight='bold')
    
    ax.set_xlabel('Number of Features', fontweight='bold')
    ax.set_ylabel('ROC-AUC', fontweight='bold')
    ax.set_title('D) Performance vs Feature Count', fontweight='bold', loc='left')
    ax.grid(True, alpha=0.3)
    
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('ROC-AUC', fontweight='bold')
else:
    ax.text(0.5, 0.5, 'Performance vs Features data not available', 
            ha='center', va='center', transform=ax.transAxes, fontsize=12)
    ax.set_title('D) Performance vs Features', fontweight='bold', loc='left')

plt.tight_layout()
plt.savefig('output/Phase_2/figure5_feature_analysis.png', dpi=300, bbox_inches='tight')
print("  Saved: output/Phase_2/figure5_feature_analysis.png")
plt.close()

if optimal_results:
    print("Creating Figure 6: Prediction Probability Distribution...")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Optimal Pipeline: Prediction Analysis', fontsize=16, fontweight='bold')
    
    predictions = np.array(optimal_results['predictions'])
    probabilities = np.array(optimal_results['probabilities'])
    true_labels = np.array(optimal_results['true_labels'])
    
    ax = axes[0, 0]
    control_probs = probabilities[true_labels == 0]
    mdd_probs = probabilities[true_labels == 1]
    
    ax.hist(control_probs, bins=20, alpha=0.6, label='True Control', color='blue', edgecolor='black')
    ax.hist(mdd_probs, bins=20, alpha=0.6, label='True MDD', color='red', edgecolor='black')
    ax.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Decision Threshold')
    ax.set_xlabel('Predicted Probability (MDD)', fontweight='bold')
    ax.set_ylabel('Frequency', fontweight='bold')
    ax.set_title('A) Probability Distribution by True Class', fontweight='bold', loc='left')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    ax = axes[0, 1]
    confidence = np.abs(probabilities - 0.5)
    correct = predictions == true_labels
    
    ax.scatter(np.arange(len(probabilities))[correct], confidence[correct], 
               c='green', label='Correct', s=60, alpha=0.6, edgecolors='black', linewidth=0.5)
    ax.scatter(np.arange(len(probabilities))[~correct], confidence[~correct], 
               c='red', label='Incorrect', s=60, alpha=0.6, edgecolors='black', linewidth=0.5)
    
    ax.axhline(y=0.2, color='orange', linestyle='--', alpha=0.5, linewidth=2, label='Low Confidence')
    ax.axhline(y=0.4, color='green', linestyle='--', alpha=0.5, linewidth=2, label='High Confidence')
    
    ax.set_xlabel('Sample Index', fontweight='bold')
    ax.set_ylabel('Prediction Confidence', fontweight='bold')
    ax.set_title('B) Prediction Confidence per Sample', fontweight='bold', loc='left')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    ax = axes[1, 0]
    sorted_idx = np.argsort(probabilities)
    sorted_probs = probabilities[sorted_idx]
    sorted_true = true_labels[sorted_idx]
    
    cumsum_true_pos = np.cumsum(sorted_true)
    cumsum_false_pos = np.cumsum(1 - sorted_true)
    
    tpr = cumsum_true_pos / cumsum_true_pos[-1] if cumsum_true_pos[-1] > 0 else cumsum_true_pos
    fpr = cumsum_false_pos / cumsum_false_pos[-1] if cumsum_false_pos[-1] > 0 else cumsum_false_pos
    
    ax.plot(fpr, tpr, linewidth=3, color='darkblue', label=f"AUC = {optimal_results['metrics']['auc']:.3f}")
    ax.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Random (AUC = 0.5)')
    ax.fill_between(fpr, tpr, alpha=0.3, color='blue')
    
    ax.set_xlabel('False Positive Rate', fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontweight='bold')
    ax.set_title('C) ROC Curve', fontweight='bold', loc='left')
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    
    ax = axes[1, 1]
    
    n_bins = 10
    bins = np.linspace(0, 1, n_bins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    binned_true = []
    binned_count = []
    
    for i in range(n_bins):
        mask = (probabilities >= bins[i]) & (probabilities < bins[i+1])
        if np.sum(mask) > 0:
            binned_true.append(np.mean(true_labels[mask]))
            binned_count.append(np.sum(mask))
        else:
            binned_true.append(np.nan)
            binned_count.append(0)
    
    valid_mask = ~np.isnan(binned_true)
    ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect Calibration')
    ax.scatter(bin_centers[valid_mask], np.array(binned_true)[valid_mask], 
               s=np.array(binned_count)[valid_mask]*10, c='purple', 
               alpha=0.6, edgecolors='black', linewidth=2, label='Observed')
    
    ax.set_xlabel('Mean Predicted Probability', fontweight='bold')
    ax.set_ylabel('Fraction of Positives', fontweight='bold')
    ax.set_title('D) Calibration Plot', fontweight='bold', loc='left')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    
    plt.tight_layout()
    plt.savefig('output/Phase_2/figure6_prediction_analysis.png', dpi=300, bbox_inches='tight')
    print("  Saved: output/Phase_2/figure6_prediction_analysis.png")
    plt.close()

print("Creating Figure 7: Comprehensive Radar Chart...")

from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

def radar_factory(num_vars, frame='circle'):
    theta = np.linspace(0, 2 * np.pi, num_vars, endpoint=False)
    
    class RadarAxes(PolarAxes):
        name = 'radar'
        
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.set_theta_zero_location('N')
        
        def fill(self, *args, closed=True, **kwargs):
            return super().fill(closed=closed, *args, **kwargs)
        
        def plot(self, *args, **kwargs):
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)
        
        def _close_line(self, line):
            x, y = line.get_data()
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)
        
        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)
        
        def _gen_axes_patch(self):
            return Circle((0.5, 0.5), 0.5)
        
        def _gen_axes_spines(self):
            spine = Spine(axes=self, spine_type='circle', path=Path.unit_circle())
            spine.set_transform(Affine2D().scale(.5).translate(.5, .5) + self.transAxes)
            return {'polar': spine}
    
    register_projection(RadarAxes)
    return theta

categories = ['Accuracy', 'ROC-AUC', 'MCC', 'Precision', 'Sensitivity', 'Specificity']
N = len(categories)

theta = radar_factory(N, frame='polygon')

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='radar'))
fig.suptitle('Multi-Metric Performance Radar Chart', fontsize=16, fontweight='bold', y=0.98)

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
color_idx = 0

if optimal_results:
    cm = optimal_results['metrics']['confusion_matrix']
    tn, fp, fn, tp = cm[0,0], cm[0,1], cm[1,0], cm[1,1]
    
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    values = [
        optimal_results['metrics']['accuracy'],
        optimal_results['metrics']['auc'],
        (optimal_results['metrics']['mcc'] + 1) / 2,
        precision,
        sensitivity,
        specificity
    ]
    values += values[:1]
    
    ax.plot(theta, values[:-1], 'o-', linewidth=2, color=colors[color_idx], 
            label='Optimal', markersize=8)
    ax.fill(theta, values[:-1], alpha=0.15, color=colors[color_idx])
    color_idx += 1

top_approaches = sorted(approach_results.items(), 
                        key=lambda x: np.mean(x[1]['roc_auc']), 
                        reverse=True)[:2]

for name, results in top_approaches:
    if color_idx >= len(colors):
        break
    
    values = [
        np.mean(results['accuracy']),
        np.mean(results['roc_auc']),
        (np.mean(results['mcc']) + 1) / 2,
        np.mean(results['accuracy']),
        np.mean(results['accuracy']),
        np.mean(results['accuracy'])
    ]
    values += values[:1]
    
    ax.plot(theta, values[:-1], 'o-', linewidth=2, color=colors[color_idx], 
            label=name, markersize=6)
    ax.fill(theta, values[:-1], alpha=0.1, color=colors[color_idx])
    color_idx += 1

ax.set_varlabels(categories)
ax.set_ylim(0, 1)
ax.grid(True)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)

plt.tight_layout()
plt.savefig('output/Phase_2/figure7_radar_chart.png', dpi=300, bbox_inches='tight')
print("  Saved: output/Phase_2/figure7_radar_chart.png")
plt.close()

print("\n" + "=" * 80)
print("VISUALIZATION SUMMARY")
print("=" * 80)
print("\nAll visualizations created successfully!\n")
print("Generated Figures:")
print("   1. figure1_performance_comparison.png     - Bar charts with error bars")
print("   2. figure2_performance_variability.png    - Box plots showing CV fold variation")
print("   3. figure3_roc_auc_confidence.png         - ROC-AUC with confidence intervals")
print("   4. figure4_confusion_matrix.png           - Confusion matrix heatmaps")
print("   5. figure5_feature_analysis.png           - Feature selection analysis (4 panels)")
print("   6. figure6_prediction_analysis.png        - Prediction distribution & ROC (4 panels)")
print("   7. figure7_radar_chart.png                - Multi-metric radar comparison")
print("\n" + "=" * 80)


CREATING VISUALIZATIONS

Loading all results...
  Approach 1 loaded
  Approach 2 loaded
  Approach 3 loaded
  Approach 4 loaded
  Approach 5 loaded
  Optimal pipeline loaded
  Final model loaded

All available data loaded

Creating Figure 1: Overall Performance Comparison...
  Saved: output/Phase_2/figure1_performance_comparison.png
Creating Figure 2: Performance Variability Across Folds...
  Saved: output/Phase_2/figure2_performance_variability.png
Creating Figure 3: ROC-AUC with Confidence Intervals...
  Saved: output/Phase_2/figure3_roc_auc_confidence.png
Creating Figure 4: Confusion Matrix Heatmap...
  Saved: output/Phase_2/figure4_confusion_matrix.png
Creating Figure 5: Feature Selection Analysis...
  Saved: output/Phase_2/figure5_feature_analysis.png
Creating Figure 6: Prediction Probability Distribution...
  Saved: output/Phase_2/figure6_prediction_analysis.png
Creating Figure 7: Comprehensive Radar Chart...
  Saved: output/Phase_2/figure7_radar_chart.png

VISUALIZATION SUMMARY